<a href="https://colab.research.google.com/github/BonoGiorgio02/6DPose_Estimation/blob/saves/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6D Pose Estimation

## Set up the project

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/6D_pose_estimation/

/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation


In [3]:
path = !pwd
path = path[0]
print(path)

/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation


In [4]:
# PRIMO BLOCCO - Solo installazioni (senza import!)
%%capture

!pip install comet_ml
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import watch

import os
import torch
os.environ['TORCH'] = torch.__version__

# Installa PRIMA tutte le dipendenze PyTorch Geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install matplotlib seaborn scikit-learn pyyaml plotly open3d Pillow ultralytics wandb numpy-quaternion trimesh

print("Installazione completata! RIAVVIA IL RUNTIME prima di procedere.")

In [5]:
import os
import yaml
import torch
import torchvision
import torchvision.models as models
import open3d as o3d
import itertools
import shutil
from torch.utils.data import Dataset
from torch import nn, optim
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import wandb
from scipy.spatial.transform import Rotation as R
from torchvision import models
import cv2
from torch.optim import Adam
import quaternion
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from ultralytics import YOLO
from torchvision.transforms import v2
import trimesh

# Import PyTorch Geometric DOPO l'installazione e il riavvio
import torch_geometric
from torch import Tensor
from torch_geometric.nn import knn_interpolate, MessagePassing
from torch_geometric.nn.pool import fps, radius

IMG_WIDTH = 640
IMG_HEIGHT = 480

# Test per verificare che tutto funzioni
try:
    from torch_geometric.nn.pool import fps
    print("✅ PyTorch Geometric installato correttamente!")
    print(f"PyTorch version: {torch.__version__}")
    print(f"PyTorch Geometric version: {torch_geometric.__version__}")
except ImportError as e:
    print(f"❌ Errore: {e}")
    print("Assicurati di aver riavviato il runtime dopo l'installazione.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ PyTorch Geometric installato correttamente!
PyTorch version: 2.6.0+cu124
PyTorch Geometric version: 2.6.1


In [6]:
# %%capture
# !pip install torch torchvision torchaudio matplotlib seaborn scikit-learn pyyaml plotly open3d pcl Pillow ultralytics wandb numpy-quaternion trimesh torch_geometric

In [7]:
# import os
# import yaml
# import torch
# import torchvision
# import open3d as o3d
# import itertools
# import shutil
# from torch.utils.data import Dataset
# from torch import nn, optim
# import torch.nn.functional as F
# from PIL import Image
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt
# import numpy as np
# import plotly.graph_objects as go
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# import wandb
# from scipy.spatial.transform import Rotation as R
# from torchvision import models
# import cv2
# from torch.optim import Adam
# import quaternion
# from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
# from tqdm import tqdm
# from torch.cuda.amp import GradScaler, autocast
# from ultralytics import YOLO
# from torchvision.transforms import v2
# import trimesh
# import torch_geometric
# from torch import Tensor
# from torch_geometric.nn import knn_interpolate, MessagePassing
# from torch_geometric.nn.pool import fps, radius

# IMG_WIDTH = 640
# IMG_HEIGHT = 480

In [8]:
# os.environ['TORCH'] = torch.__version__

In [9]:
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [10]:
if torch.cuda.is_available():
    print("Cuda")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Cuda not available, use mps")
    device = torch.device("mps")
else:
    print("Use CPU")
    device = torch.device("cpu")

Cuda


## Download dataset

We will work with a portion of this dataset, which you can find here: https://drive.google.com/drive/folders/19ivHpaKm9dOrr12fzC8IDFczWRPFxho7

In [11]:
!cp /content/drive/MyDrive/6D_pose_estimation/datasets/linemod/DenseFusion/Linemod_preprocessed_zip/Linemod_preprocessed.zip /content/

In [12]:
!mkdir -p /content/datasets/linemod/DenseFusion/
%cd /content/datasets/linemod/DenseFusion/

/content/datasets/linemod/DenseFusion


In [13]:
%%capture
!unzip /content/Linemod_preprocessed.zip -d /content/datasets/linemod/DenseFusion/

## Modify the dataset

In [14]:
for el in ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]:
    shutil.copy(f"./Linemod_preprocessed/data/{el}/gt.yml", f"./Linemod_preprocessed/{el}_gt.yml")

In [15]:
def change_02gt(path):
    with open(path, 'r') as f:
        pose_data = yaml.load(f, Loader=yaml.CLoader)  # Usiamo safe_load per maggiore sicurezza

    filtered_data = {}
    for key, value in pose_data.items():
        filtered_frame = [obj for obj in value if int(obj['obj_id']) == 2]
        if filtered_frame:
            filtered_data[key] = filtered_frame

    # Custom Dumper per mantenere il formato originale
    class OriginalFormatDumper(yaml.Dumper):
        def increase_indent(self, flow=False, indentless=False):
            return super().increase_indent(flow, False)  # Forza indentazione corretta

    # Configurazione per mantenere il formato desiderato
    yaml_str = yaml.dump(
        filtered_data,
        sort_keys=True,
        default_flow_style=None,  # None = auto (usa flow-style per liste interne)
        Dumper=OriginalFormatDumper,
        width=float("inf"),
        indent=2,  # Indentazione standard
        allow_unicode=True,
    )

    # Salva il risultato nel file
    with open(path, 'w') as f:
        yaml.dump(
            filtered_data,
            f,
            sort_keys=True,
            default_flow_style=None,
            Dumper=OriginalFormatDumper,
            width=float("inf"),
            indent=2,
        )

change_02gt("./Linemod_preprocessed/02_gt.yml")

In [16]:
def quaternion_gt(input_path= "./Linemod_preprocessed/"):

  for gt in os.scandir(input_path):
        if not gt.is_dir() and gt.name.endswith(('.yaml', '.yml')):

          with open(gt, 'r') as f:
            pose_data = yaml.load(f, Loader=yaml.CLoader)
            modified_data = {}

            for key,value in pose_data.items():
              modified_poses = []

              for pose in value:
                quat= quaternion.from_rotation_matrix(np.array(pose["cam_R_m2c"]).reshape(3,3))
                pose["quaternion"]= [quat.w, quat.x, quat.y, quat.z]
                modified_poses.append(pose)
              modified_data[key] = modified_poses

            output_file = os.path.join(input_path, gt.name)

            with open(gt, 'w') as f:
                yaml.dump(
                    modified_data,
                    f,
                    default_flow_style=None,
                    width=float("inf"),
                    sort_keys=True
                )
quaternion_gt()

## Data Exploration

### Custom Dataset

In [ ]:
img_path = "./Linemod_preprocessed/data/02/rgb/0101.png"
img = Image.open(img_path).convert("RGB")
plt.imshow(img)
plt.show()

In [ ]:
class CustomDataset(Dataset): # used to load and preprocess data
    def __init__(self, dataset_root, split='train', train_ratio=0.7, seed=42):
        """
        Args:
            dataset_root (str): Path to the dataset directory.
            split (str): 'train', 'validation' or 'test'.
            train_ratio (float): Percentage of data used for training (default 70%).
            seed (int): Random seed for reproducibility.
        """
        self.dataset_root = dataset_root
        self.split = split
        self.train_ratio = train_ratio
        self.seed = seed

        # Get list of all samples (folder_id, sample_id)
        self.samples = self.get_all_samples()

        # Check if samples were found
        if not self.samples:
            raise ValueError(f"No samples found in {self.dataset_root}. Check the dataset path and structure.")

        # Split into training and validation+test sets
        labels = [el[0] for el in self.samples]
        self.train_samples, self.val_test_samples = train_test_split(
            self.samples, train_size=self.train_ratio, random_state=self.seed, stratify=labels
        )

        # split validation+test set (by default 30% of the original dataset) into validation and test sets
        labels = [el[0] for el in self.val_test_samples]
        self.val_samples, self.test_samples = train_test_split(self.val_test_samples, train_size=0.5, random_state=self.seed, stratify=labels)

        # Select the appropriate split
        if split == "train":
            self.samples = self.train_samples
        elif split == "validation":
            self.samples = self.val_samples
        else:
            self.samples = self.test_samples

        # Define image transformations: QUESTO PER LA BASELINE
        if self.split == 'train':
            # Define image transformations
        #     self.transform_img = v2.Compose([
        #       # v2.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05),
        #       # v2.RandomGrayscale(p=0.1),
        #       # v2.RandomApply([v2.GaussianBlur(kernel_size=3)], p=0.1),
        #       # v2.ToImage(),  # Converte in PILImage o equivalente
        #       v2.ToDtype(torch.float32),  # Converte in Tensor con valori tra 0 e 1
        #       v2.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
        #   ])
        #     self.transform_crop = v2.Compose([
        #       # v2.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05),
        #       # v2.RandomGrayscale(p=0.1),
        #       # v2.RandomApply([v2.GaussianBlur(kernel_size=3)], p=0.1),
        #       v2.Resize((224, 224)),
        #       v2.ToImage(),  # Converte in PILImage o equivalente
        #       v2.ToDtype(torch.float32),  # Converte in Tensor con valori tra 0 e 1
        #       v2.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
        #   ])

        # else:
        #     self.transform_img = v2.Compose([
        #         v2.ToImage(),  # Converte in PILImage o equivalente
        #         v2.ToDtype(torch.float32),  # Converte in Tensor con valori tra 0 e 1
        #         v2.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
        #     ])

        #     self.transform_crop = v2.Compose([
        #         v2.Resize((224, 224)),
        #         v2.ToImage(),  # Converte in PILImage o equivalente
        #         v2.ToDtype(torch.float32),  # Converte in Tensor con valori tra 0 e 1
        #         v2.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
        #     ])

            self.transform_img = transforms.Compose([
                                # transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05),
                                # transforms.RandomGrayscale(p=0.1),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.1),
                                transforms.ToTensor(),  # converte in float32 e normalizza in [0, 1]
                                # transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
                            ])

            self.transform_crop = transforms.Compose([
                                # transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05),
                                # transforms.RandomGrayscale(p=0.1),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.1),
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
    ])
        else:
              self.transform_img = transforms.Compose([
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
                            ])

              self.transform_crop = transforms.Compose([
                  transforms.Resize((224, 224)),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
              ])

        self.ground_truths = {}

        for el in ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]:

          pose_file = os.path.join(self.dataset_root, f"{el}_gt.yml")

          with open(pose_file, 'r') as f:
            pose_data = yaml.load(f, Loader=yaml.CLoader)

          dati = {}
          chiavi_da_estrarre = ['cam_t_m2c', 'cam_R_m2c', 'quaternion', 'obj_bb', 'obj_id']
          dati_estratti = {}

          for key, value in pose_data.items():
            entry = value[0]

            # Estraggo solo le chiavi desiderate
            estratti = {k: entry[k] for k in chiavi_da_estrarre if k in entry}

            # Salvo nel dizionario dei dati estratti
            dati_estratti[key]= estratti
          self.ground_truths[el] = dati_estratti




    def get_samples_id(self):
        return self.samples

    def get_all_samples(self):
        """Retrieve the list of all available sample indices from all folders."""
        samples = []
        for folder_id in range(1, 16):  # Assuming folders are named 01 to 15
            folder_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", "rgb")
            #print(folder_path)
            if os.path.exists(folder_path):
                # get id of the images
                sample_ids = sorted([int(f.split('.')[0]) for f in os.listdir(folder_path) if f.endswith('.png')])
                samples.extend([(folder_id, sid) for sid in sample_ids])  # Store (folder_id, sample_id)
        return samples

    def load_config(self, folder_id):
        """Load YAML configuration files for camera intrinsics and object info for a specific folder."""
        # camera_intrinsics_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", 'info.yml')
        objects_info_path = os.path.join(self.dataset_root, 'models', f"models_info.yml")

        # with open(camera_intrinsics_path, 'r') as f:
            # camera_intrinsics = yaml.load(f, Loader=yaml.FullLoader)
        camera_intrinsics = [572.4114, 573.57043, 325.2611, 242.04899]

        with open(objects_info_path, 'r') as f:
            objects_info = yaml.load(f, Loader=yaml.CLoader)

        return camera_intrinsics, objects_info

    #Define here some usefull functions to access the data
    def load_image(self, img_path):
        """Load an RGB image and convert to tensor."""
        img = Image.open(img_path).convert("RGB")
        return self.transform_img(img)

    #Define here some usefull functions to access the data
    def load_cropped_image(self, img_path, bbox):
        """Load an RGB image and convert to tensor."""
        img = Image.open(img_path).convert("RGB")
        x, y, w, h = bbox
        cropped_img = img.crop((x, y, x+w, y+h))
        return self.transform_crop(cropped_img)

    def load_depth(self, depth_path):
        """Load a depth image and convert to tensor."""
        return cv2.imread(depth_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

    def load_mask(self, path):
        return cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.uint8)

    def depth_to_pointcloud(self, masked_depth, intrinsics):
        """
        Convert pixel coordinates + depth to 3D metric coordinates using intrinsic parameters.
        Returns x, y, z as Nx3 array.
        """
        fx, fy, cx, cy = intrinsics
        height, width = masked_depth.shape
        valid_mask = masked_depth > 0

        u, v = np.meshgrid(np.arange(width), np.arange(height))
        u_valid = u[valid_mask]
        v_valid = v[valid_mask]
        z_valid = masked_depth[valid_mask].astype(np.float32) / 1000.0

        # Ensure z_valid is not zero or very small to avoid division by zero
        valid_depth_mask = z_valid > 1e-6
        u_valid = u_valid[valid_depth_mask]
        v_valid = v_valid[valid_depth_mask]
        z_valid = z_valid[valid_depth_mask]

        x_meters = (u_valid - cx) * z_valid / fx
        y_meters = (v_valid - cy) * z_valid / fy
        z_meters = z_valid

        pointcloud = np.stack([x_meters, y_meters, z_meters], axis=-1).reshape(-1, 3)  # Shape: (N, 3)
        return pointcloud

    def load_6d_pose(self, folder_id, sample_id):
        """Load the 6D pose (translation and rotation) for the object in this sample."""
        pose = self.ground_truths[f"{folder_id:02d}"][int(sample_id)]
        # print(self.traslations[f"{folder_id:02d}"][f"{sample_id:04d
        # pose_file = os.path.join(self.dataset_root, f"{folder_id:02d}_gt.yml")

        # Load the ground truth poses from the gt.yml file
        # with open(pose_file, 'r') as f:
            # pose_data = yaml.load(f, Loader=yaml.CLoader)

        # The pose data is a dictionary where each key corresponds to a frame with pose info
        # We assume sample_id corresponds to the key in pose_data
        # if sample_id not in pose_data:
            # raise KeyError(f"Sample ID {sample_id} not found in gt.yml for folder {folder_id}.")

        # for pose in pose_data[sample_id]: # There can be more than one pose per sample, but take the one of label=folder_id
            # Extract translation and rotation
            # if (int(pose['obj_id']) == int(folder_id)):
        translation = np.array(pose['cam_t_m2c'], dtype=np.float32)/1000.0  # [3] ---> (x,y,z)
        rotation = np.array(pose['cam_R_m2c'], dtype=np.float32).reshape(3, 3)  # [3x3] ---> rotation matrix
        quaternion = np.array(pose['quaternion'], dtype=np.float32)  # [4] ---> quaternion
        bbox_base = np.array(pose['obj_bb'], dtype=np.float32) # [4] ---> x_min, y_min, width, height
        # bbox is top left corner and width and height info, YOLO needs center coordinates and width and height
        # x_min, y_min, width, height = np.array(pose['obj_bb'], dtype=np.float32) # [4] ---> x_min, y_min, width, height
        cropped_img = self.load_cropped_image(os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", "rgb", f"{sample_id:04d}.png"), np.array(pose['obj_bb'], dtype=np.float32))

                # Compute initial center
                # x_center = x_min + width / 2
                # y_center = y_min + height / 2

                # Clip center to image bounds and adjust width/height accordingly
                # if x_center < 0:
                #     width += 2 * x_center  # x_center is negative, subtract its absolute value * 2 from width
                #     x_center = 0
                # elif x_center > IMG_WIDTH:
                #     width -= 2 * (x_center - IMG_WIDTH)
                #     x_center = IMG_WIDTH

                # # if y_center<0 take the max(0,y_center), if y_center>IMG_WIDTH take min(IMG_WIDTH, y_center)
                # if y_center < 0:
                #     height += 2 * y_center
                #     y_center = 0
                # elif y_center > IMG_HEIGHT:
                #     height -= 2 * (y_center - IMG_HEIGHT)
                #     y_center = IMG_HEIGHT

                # Ensure width and height are not negative
                # Caso in cui la bounding box sia completamente al di fuori dell'immagine (non dovrebbe mai capitare)
                # width = max(0, width)
                # height = max(0, height)
                # store coordinates of the center and width and height of the bounding box normalized to the
                # image width=640 pixels and height=480 pixels
                # bbox_YOLO = np.array([x_center/IMG_WIDTH, y_center/IMG_HEIGHT, width/IMG_WIDTH, height/IMG_HEIGHT], dtype=np.float32)

        obj_id = np.array(pose['obj_id'], dtype=np.float32) # [1] ---> label
                # break
        # BISOGNA AGGIUNGERE bbox_YOLO
        return cropped_img, translation, rotation, quaternion, bbox_base, obj_id

    def __len__(self):
        #Return the total number of samples in the selected split.
        return len(self.samples)

    def __getitem__(self, idx):
        #Load a dataset sample.
        folder_id, sample_id = self.samples[idx]

        # Load the correct camera intrinsics and object info for this folder
        # camera_intrinsics, objects_info = self.load_config(folder_id)

        img_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"rgb/{sample_id:04d}.png")
        # depth_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"depth/{sample_id:04d}.png")
        # mask_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"mask/{sample_id:04d}.png")

        img = self.load_image(img_path)
        # depth = self.load_depth(depth_path)
        # mask = self.load_mask(mask_path)

        # mask_binary = mask != 0
        # masked_depth = np.where(mask_binary, depth, 0)

        # pointcloud = self.depth_to_pointcloud(masked_depth, camera_intrinsics),

        # BISOGNA AGGIUNGERE bbox_YOLO
        cropped_img, translation, rotation, quaternion, bbox_base, obj_id = self.load_6d_pose(folder_id, sample_id)

        #Dictionary with all the data
        return {
            "rgb": img,
            "cropped_img": cropped_img.to(device),
            # "depth": torch.tensor(depth, dtype=torch.float32),
            # "pointcloud": torch.tensor(pointcloud, dtype=torch.float32),
            # "camera_intrinsics": camera_intrinsics,
            # "objects_info": objects_info,
            "translation": torch.tensor(translation).to(device),
            "rotation": torch.tensor(rotation).to(device),
            "quaternion": torch.tensor(quaternion).to(device),
            "bbox_base": torch.tensor(bbox_base).to(device),
            # "bbox_YOLO": torch.tensor(bbox_YOLO),
            "obj_id": torch.tensor(obj_id).to(device),
            "sample_id": torch.tensor(self.samples[idx]).to(device)
        }

In [ ]:
dataset_root = "./Linemod_preprocessed/"

train_dataset = CustomDataset(dataset_root, split="train")
print(f"Training samples: {len(train_dataset)}")

val_dataset = CustomDataset(dataset_root, split="validation")
print(f"Validation samples: {len(val_dataset)}")

test_dataset = CustomDataset(dataset_root, split="test")
print(f"Testing samples: {len(test_dataset)}")

### Visualization 2D - 3D - 2D con y e y ribaltata

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

depth_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data\\01\\depth\\0000.png'
mask_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data\\01\\mask\\0000.png'

# Parametri intrinseci della camera LineMOD (dai tuoi dati)
fx = 572.4114  # focal length x
fy = 573.57043  # focal length y
cx = 325.2611  # principal point x
cy = 242.04899  # principal point y

# Carica la depth map come immagine a 16 bit
depth_img = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)  # mantiene dtype uint16
mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask_binary = mask_img != 0

masked_depth = np.where(mask_binary, depth_img, 0)

print("Shape:", depth_img.shape)
print("Dtype:", depth_img.dtype)
print("Min depth:", np.min(depth_img))
print("Max depth:", np.max(depth_img))
print("Pixels = 0:", np.sum(depth_img == 0))
print("Total pixels:", 480*640)

def pixel_to_camera_coordinates(masked_depth, fx, fy, cx, cy):
    """
    Converte coordinate pixel + depth in coordinate metriche 3D usando i parametri intrinseci

    Formula di conversione:
    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy
    Z = depth

    dove (u,v) sono coordinate pixel e Z è la profondità
    """
    height, width = masked_depth.shape

    # Trova tutti i pixel con profondità valida (non zero)
    valid_mask = masked_depth > 0

    # Crea griglia di coordinate pixel
    u, v = np.meshgrid(np.arange(width), np.arange(height))

    # Estrai coordinate dei pixel validi
    u_valid = u[valid_mask]  # coordinate u (pixel)
    v_valid = v[valid_mask]  # coordinate v (pixel)
    z_valid = masked_depth[valid_mask].astype(np.float32) / 1000.0  # profondità in metri

    # Conversione da coordinate pixel a coordinate camera (tutte in metri)
    x_meters = (u_valid - cx) * z_valid / fx
    y_meters = (v_valid - cy) * z_valid / fy
    z_meters = z_valid

    return x_meters, y_meters, z_meters, u_valid, v_valid

# Crea la point cloud 3D in coordinate metriche
x_meters, y_meters, z_meters, u_pixels, v_pixels = pixel_to_camera_coordinates(masked_depth, fx, fy, cx, cy)

print(f"\n=== Point Cloud 3D Info (Coordinate Metriche) ===")
print(f"Numero di punti validi: {len(x_meters)}")
print(f"Range X (metri): {x_meters.min():.3f} - {x_meters.max():.3f}")
print(f"Range Y (metri): {y_meters.min():.3f} - {y_meters.max():.3f}")
print(f"Range Z (metri): {z_meters.min():.3f} - {z_meters.max():.3f}")

print(f"\n=== Parametri Camera ===")
print(f"fx: {fx}, fy: {fy}")
print(f"cx: {cx}, cy: {cy}")
print(f"Dimensioni immagine: {masked_depth.shape[1]}x{masked_depth.shape[0]} pixel")

# === Visualizzazione Comparativa ===
fig = plt.figure(figsize=(12, 8))

# Immagine depth originale
plt.subplot(2, 2, 1)
plt.title("Depth Original", fontsize=12)
plt.imshow(depth_img, cmap='gray')
plt.colorbar(label='Depth (mm)')
plt.xlabel('X (pixel)')
plt.ylabel('Y (pixel)')

# Masked Depth
plt.subplot(2, 2, 2)
plt.title("Masked Depth", fontsize=12)
plt.imshow(masked_depth, cmap='gray')
plt.colorbar(label='Depth (mm)')
plt.xlabel('X (pixel)')
plt.ylabel('Y (pixel)')

# Distribuzione punti in coordinate pixel (per confronto)
plt.subplot(2, 2, 3)
plt.title("Point Distribution (Pixel Coords)", fontsize=12)
scatter_pixel = plt.scatter(u_pixels, v_pixels, c=z_meters, cmap='viridis', s=6, alpha=0.6)
plt.colorbar(scatter_pixel, label='Depth (m)')
plt.xlabel('X (pixel)')
plt.ylabel('Y (pixel)')
plt.gca().invert_yaxis()

# === COORDINATE METRICHE (LA PARTE IMPORTANTE) ===

# Plot 3D principale - COORDINATE METRICHE
ax1 = fig.add_subplot(2, 2, 4, projection='3d')
ax1.set_title("3D Point Cloud\n(X,Y,Z tutti in metri)", fontsize=12)
scatter3d = ax1.scatter(x_meters, y_meters, z_meters, c=z_meters, cmap='plasma', s=1, alpha=0.7)
ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_zlabel('Z (m)')
plt.colorbar(scatter3d, ax=ax1, shrink=0.5, aspect=5, label='Depth (m)')

plt.tight_layout()
plt.show()

# === Visualizzazione 3D dettagliata con coordinate metriche ===
fig = plt.figure(figsize=(16, 12))

# Plot 3D con diversi angoli di vista
angles = [(30, 45), (60, 30), (0, 0), (90, 90)]
titles = ["3D View (30°, 45°)", "3D View (60°, 30°)", "Front View (X-Y)", "Top View (X-Z)"]

for i, (elev, azim) in enumerate(angles):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    scatter = ax.scatter(x_meters, y_meters, z_meters, c=z_meters, cmap='plasma', s=0.8, alpha=0.7)
    ax.set_title(f"{titles[i]} - Metric Coordinates", fontsize=12)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.view_init(elev=elev, azim=azim)

    if i == 0:  # Aggiungi colorbar solo al primo plot
        plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5, label='Depth (m)')

plt.tight_layout()
plt.show()

# === Salva i dati ===
# Salva la point cloud in coordinate metriche
point_cloud_metric = np.column_stack((x_meters, y_meters, z_meters))
point_cloud_pixel = np.column_stack((u_pixels, v_pixels, z_meters))

np.save('masked_depth_3d_metric_coords.npy', point_cloud_metric)
np.save('masked_depth_3d_pixel_coords.npy', point_cloud_pixel)

print(f"\n=== Salvataggio ===")
print(f"Point cloud METRICA salvata in 'masked_depth_3d_metric_coords.npy'")
print(f"Point cloud PIXEL salvata in 'masked_depth_3d_pixel_coords.npy'")
print(f"Shape metrica: {point_cloud_metric.shape}")
print(f"Shape pixel: {point_cloud_pixel.shape}")
print(f"Formato metrico: [x_metri, y_metri, z_metri]")
print(f"Formato pixel: [x_pixel, y_pixel, z_metri]")

# Mostra alcuni esempi di punti
print(f"\nPrimi 10 punti (COORDINATE METRICHE):")
print(f"[X_m, Y_m, Z_m]")
for i in range(min(10, len(point_cloud_metric))):
    x, y, z = point_cloud_metric[i]
    print(f"[{x:+.3f}, {y:+.3f}, {z:.3f}]")

print(f"\nPrimi 10 punti (COORDINATE PIXEL + Z metri):")
print(f"[X_px, Y_px, Z_m]")
for i in range(min(10, len(point_cloud_pixel))):
    x, y, z = point_cloud_pixel[i]
    print(f"[{x:3.0f}, {y:3.0f}, {z:.3f}]")

# Calcola e mostra il centroide dell'oggetto
centroid = np.mean(point_cloud_metric, axis=0)
print(f"\nCentroide dell'oggetto (coordinate metriche):")
print(f"X: {centroid[0]:.3f} m")
print(f"Y: {centroid[1]:.3f} m")

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Carica la point cloud in coordinate metriche
# Se hai già salvato la point cloud
# point_cloud_metric = np.load('masked_depth_3d_metric_coords.npy')

# Altrimenti puoi usare direttamente x_meters, y_meters, z_meters
point_cloud_metric = np.column_stack((x_meters, y_meters, z_meters))

# Crea un oggetto PointCloud di open3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud_metric)

# (Opzionale) Aggiungi colore in base alla profondità (Z)
colors = (z_meters - z_meters.min()) / (z_meters.max() - z_meters.min())
colors = plt.cm.plasma(colors)[:, :3]  # Usa colormap matplotlib, prendi solo RGB
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualizza la point cloud con viewer 3D interattivo
o3d.visualization.draw_geometries([pcd], window_name="3D Point Cloud")

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

depth_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data\\01\\depth\\0000.png'
rgb_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data\\01\\rgb\\0000.png'
mask_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data\\01\\mask\\0000.png'

# === Carica dati ===
# (Modifica il path per caricare la tua immagine e profondità)
rgb = cv2.imread(rgb_path)
rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 1000.0  # in metri
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

# Carica parametri intrinseci della camera
fx, fy = 572.4114, 573.57043
cx, cy = 325.2611, 242.04899

# === Estrai pixel validi ===
v, u = np.where(mask > 0)
z = depth[v, u]
x = (u - cx) * z / fx
y = (v - cy) * z / fy

# === Point cloud senza inversione Y ===
pc_natural = np.column_stack((x, y, z))

# === Point cloud con inversione Y ===
y_inv = -y
pc_inverted = np.column_stack((x, y_inv, z))

# === Proiezione 3D -> 2D ===
def project_to_2d(x, y, z):
    u_proj = (x * fx) / z + cx
    v_proj = (y * fy) / z + cy
    return u_proj, v_proj

u_nat, v_nat = project_to_2d(pc_natural[:,0], pc_natural[:,1], pc_natural[:,2])
u_inv, v_inv = project_to_2d(pc_inverted[:,0], pc_inverted[:,1], pc_inverted[:,2])

# === Visualizza i risultati ===
fig, axs = plt.subplots(1, 3, figsize=(15,5))

# Immagine originale
axs[0].imshow(rgb)
axs[0].set_title('Immagine RGB Originale')
axs[0].axis('off')

# Proiezione senza inversione
axs[1].imshow(rgb)
axs[1].scatter(u_nat, v_nat, s=1, c='red')
axs[1].set_title('Proiezione senza inversione Y')
axs[1].axis('off')

# Proiezione con inversione Y
axs[2].imshow(rgb)
axs[2].scatter(u_inv, v_inv, s=1, c='blue')
axs[2].set_title('Proiezione con inversione Y')
axs[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
root_path = r'.\\datasets\\linemod\\DenseFusion\\Linemod_preprocessed\\data'

def extract_cam_K_from_info(info):
    # info è un dict con chiavi come '0', '1', ... ciascuna con un dict che contiene 'cam_K'
    cam_K_list = []
    for key, val in info.items():
        if "cam_K" in val:
            cam_K_list.append(val["cam_K"])
    return cam_K_list

all_cam_Ks = []  # lista di tutte le cam_K trovate in tutti i file

for obj_class in sorted(os.listdir(root_path)):
    class_path = os.path.join(root_path, obj_class)
    if not os.path.isdir(class_path):
        continue

    info_path = os.path.join(class_path, "info.yml")
    if not os.path.exists(info_path):
        continue

    with open(info_path, 'r') as f:
        info = yaml.safe_load(f)

    cam_Ks = extract_cam_K_from_info(info)
    if len(cam_Ks) == 0:
        print(f"[Warning] Nessun cam_K trovato in {obj_class}/info.yml")
        continue

    # Aggiungi tutti i cam_K di questo file alla lista globale
    all_cam_Ks.extend(cam_Ks)

# Ora controlla che tutti i cam_K siano identici
if len(all_cam_Ks) == 0:
    print("Nessun cam_K trovato in nessun file.")
else:
    first_cam_K = all_cam_Ks[0]
    all_equal = all(cam_K == first_cam_K for cam_K in all_cam_Ks)

    if all_equal:
        print("Tutti i cam_K sono uguali in tutti i file info.yml")
    else:
        print("Ci sono cam_K diversi tra i file info.yml")

## Data Preprocessing

Structure the data such that
```
datasets/
├── data.yaml
│
├── train/
│   ├── images/
│   │
│   └── labels/
│  
├── val/
│
└── test/
```

In [ ]:
# divide the dataset into training, validation and testing set
train_samples = train_dataset.get_samples_id()
validation_samples = val_dataset.get_samples_id()
test_samples = test_dataset.get_samples_id() # test folder is optional for training YOLO

Create a new folder containing all the info, we just need the rgb image and a text file with the label and bounding box.
The ```Linemod_preprocessed``` is not removed, as it contains info about translation and rotation that are needed for pose estimation, but not for object detection model.

The working directory is in the ```DenseFusion```

In [ ]:
# create a folder to contain the dataset for YOLO model
os.makedirs("./datasets/linemod/YOLO/datasets", exist_ok=True)

# count number of distinct classes
number_classes = 0
class_names = []
for el in os.scandir("./datasets/linemod/DenseFusion/Linemod_preprocessed/data"):
    # if entry is a directory and its name is an integer value (this is just to avoid counting non directories or other directories)
    if (el.is_dir() and el.name.isdigit()):
        class_names.append(el.name)
        number_classes += 1

# get string of all class names
class_names.sort() # sort the names
names = "["
for index, el in enumerate(class_names):
    # if last element don't add comma
    if index == number_classes-1:
        names += f"'{str(el)}'"
    else:
        names += f"'{str(el)}',"
names += "]"

# create data.yaml (as class names use ids of the folder)
content = f"""train: ./train/images
val: ./val/images
test: ./test/images

nc: {number_classes}
names: {names}"""
# write to file
with open("./datasets/linemod/YOLO/datasets/data.yaml", "w") as fout:
    fout.write(content)
fout.close()

While creating the folder structure, we have to change the class id by using the index in the array written in the ```data.yaml```

In [ ]:
# create a dictionary to have easily access to the index
index_dict = dict()
for index, el in enumerate(class_names):
    index_dict[int(el)] = index

Create the folders. Note that each image may contain multiple objects. For instance in ```data/02/gt.yml``` for one image there are multiple objects

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os

# create images and labels
# dataset = [train_samples, validation_samples, test_samples]
folder_names = ["train", "val", "test"]

# count also the number of instances of each class
classes = range(0, number_classes)
counter_df = pd.DataFrame()
for idx in range(3):
    if idx == 0:
        dataset = train_samples
    elif idx == 1:
        dataset = validation_samples
    else:
        dataset = test_samples
    print(f"------------------------------{folder_names[idx].upper()}------------------------------")
    os.makedirs(f"../YOLO/datasets/{folder_names[idx]}/images", exist_ok=True)
    os.makedirs(f"../YOLO/datasets/{folder_names[idx]}/labels", exist_ok=True)
    classCount = {temp_class: 0 for temp_class in range(0,number_classes)} # initialize dictionary for counting
    total = 0 # used to normalize count
    for el in tqdm(dataset, desc="Moving..."):
        if int(el[0]) == 2:
            # el is (folderId, sampleId)
            _, _, bbox, obj_id = train_dataset.load_6d_pose(el[0], el[1])
            # copy image into the new folder
            # avoid overwriting the files, so concat also the name of the folderId to the destination file
            shutil.copy(f"./Linemod_preprocessed/data/{el[0]:02d}/rgb/{el[1]:04d}.png", f"../YOLO/datasets/{folder_names[idx]}/images/{el[0]:02d}_{el[1]:04d}.png")
            # create label file with the same name as the image
            with open(f"../YOLO/datasets/{folder_names[idx]}/labels/{el[0]:02d}_{el[1]:04d}.txt", "w") as fout:
                # bbox is a list of values in the form of [x_center, y_center, width, height] and obj_id a list of class labels
                # where each label is in the format 01-15
                content = ""
                classCount[index_dict[int(obj_id)]] += 1
                total += 1
                content += f"{index_dict[int(obj_id)]} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n"
                fout.write(content)
            fout.close()

    # store in the dataframe
    values = pd.array(list(classCount.values()))/total
    counter_df[folder_names[idx]] = values.copy()

# plot distribution of labels in training, validation and test set
plt.grid(alpha=0.5)
sns.lineplot(data=counter_df)
plt.xlabel("Class")
plt.ylabel("Frequency")
plt.xticks(range(0,number_classes))
plt.savefig(f"../../../images/YOLO_dataset_distribution.png")
plt.show()

### Visualize data

Visualize depth image

In [ ]:
img_path = "./Linemod_preprocessed/data/02/depth/0101.png"
img = Image.open(img_path)
plt.imshow(img)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# Plot image with bounding box

# Load the ground truth poses from the gt.yml file
with open("./Linemod_preprocessed/data/02/gt.yml", 'r') as f:
  pose_data = yaml.load(f, Loader=yaml.FullLoader)
pose = pose_data[101][6] # access image 0 (start counting from 0) and get first object in that image (in case of multiple objects)

bbox = np.array(pose['obj_bb'], dtype=np.float32) #[4]
obj_id = np.array(pose['obj_id'], dtype=np.float32) #[1]

fig, ax = plt.subplots()
ax.imshow(img)

# Create a rectangle patch
rect = patches.Rectangle(
    (bbox[0], bbox[1]),  # (x, y)
    bbox[2],             # width
    bbox[3],             # height
    linewidth=2,
    edgecolor='red',
    facecolor='none'
)

# Add the rectangle to the plot
ax.add_patch(rect)

# Optionally add object ID label (write a bit above the top left corner)
ax.text(bbox[0], bbox[1] - 20, f'ID: {int(obj_id)}', color='yellow', fontsize=12, backgroundcolor='black')

plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# Plot image with bounding box

# Load the ground truth poses from the gt.yml file
with open("../YOLO/datasets/val/labels/02_0101.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
        parts = line.split(" ")
        bbox = np.array(parts[1:5], dtype=np.float32) #[4]
        obj_id = np.array(parts[0], dtype=np.float32) #[1]
        if obj_id == 7.0:
            x_center, y_center, width, height = bbox[0:4]
            x_center = x_center*IMG_WIDTH
            y_center = y_center*IMG_HEIGHT
            width = width*IMG_WIDTH
            height = height*IMG_HEIGHT
            x_min = x_center-(width/2)
            y_min = y_center-(height/2)
            fig, ax = plt.subplots()
            ax.imshow(img)

            # Create a rectangle patch
            rect = patches.Rectangle(
                (x_min, y_min),  # (x, y)
                width,             # width
                height,             # height
                linewidth=2,
                edgecolor='red',
                facecolor='none'
            )

            # Add the rectangle to the plot
            ax.add_patch(rect)

            # Optionally add object ID label (write a bit above the top left corner)
            ax.text(x_min, y_min - 20, f'ID: {int(obj_id)}', color='yellow', fontsize=10, backgroundcolor='black')

    plt.axis('off')
    plt.show()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
print(f"Training loader: {len(train_loader)}")
print(f"Validation loader: {len(val_loader)}")
print(f"Test loader: {len(test_loader)}")

In [ ]:
import itertools

# Get only the first 1 batch
train_subset_num_batches = 1
val_subset_num_batches = 1
test_subset_num_batches = 1
train_subset = list(itertools.islice(train_loader, train_subset_num_batches))
val_subset = list(itertools.islice(val_loader, val_subset_num_batches))
test_subset = list(itertools.islice(test_loader, test_subset_num_batches))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Get one batch from the train loader (4 images)
batch = next(iter(train_loader)) # it uses load_6d_pose, so one pose per object

# Extract relevant data
rgb_images = batch["rgb"]         # (B, 3, H, W)
bboxes = batch["bbox"]            # (B, 4) in pixel coords: x_min, y_min, x_max, y_max
obj_ids = batch["obj_id"]         # (B,)

# Convert to numpy and rearrange channels
rgb_images = rgb_images.permute(0, 2, 3, 1).numpy()  # (B, H, W, 3)
bboxes = bboxes.numpy()
obj_ids = obj_ids.numpy()

# Plot settings
batch_size = rgb_images.shape[0]
cols = min(4, batch_size)
rows = (batch_size + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(12, 3 * rows))
axes = axes.flatten()

for i in range(batch_size):
    ax = axes[i]
    img = rgb_images[i]
    # each element is [x_center/IMG_WIDTH, y_center/IMG_HEIGHT, width/IMG_WIDTH, height/IMG_HEIGHT]
    x_center, y_center, width, height = bboxes[i]
    # remove normalization
    x_center = x_center*IMG_WIDTH
    y_center = y_center*IMG_HEIGHT
    width = width*IMG_WIDTH
    height = height*IMG_HEIGHT
    x_min = x_center-(width/2)
    y_min = y_center-(height/2)
    obj_id = obj_ids[i]

    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"Sample {i}")

    # Draw bounding box
    rect = patches.Rectangle(
        (x_min, y_min),   # (x_min, y_min)
        width,              # width
        height,              # height
        linewidth=2,
        edgecolor='red',
        facecolor='none'
    )
    ax.add_patch(rect)

    # Add object ID as label
    ax.text(
        x_min,
        y_min - 10,
        f'ID: {int(obj_id)}',
        color='yellow',
        fontsize=10,
        backgroundcolor='black'
    )

# Hide unused axes if batch_size < cols * rows
for j in range(batch_size, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# !pwd
# %cd ../../test/labels

In [ ]:
# path = !pwd
# %cd datasets/linemod/YOLO/datasets/test/labels/

In [ ]:
# total = 0
# for l in os.listdir(path):
    # with open(l, 'r') as f:
        # lines = f.readlines()
        # for line in lines:
            # parts = line.split(" ")
            # x_center = parts[1]
            # y_center = parts[2]
            # if float(x_center) == 0.0 or float(y_center) == 0.0 or float(x_center) == 1.0 or float(y_center) == 1.0:
                # print(l)
                # total += 1
                # break
# print(total)

# Baseline

## Training Object Detection model

Check if CUDA available, otherwise try with MPS and then CPU

In [ ]:
# %cd ./drive/MyDrive/6D_pose_estimation/

/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation


In [ ]:
%cd ../../..

/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation


In [ ]:
old_path = !pwd
path = old_path[0]
path

'/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation'

In [ ]:
# Ricordarsi di cambiare il yaml e il percorso interno in base all'account in cui siete
# with open("./datasets/linemod/YOLO/datasets/data.yaml","r") as f:
  # f= f.read()
  # print(f)

train: ./train/images
val: ./val/images
test: ./test/images

nc: 13
names: ["01","02","04","05","06","08","09","10","11","12","13","14","15"]


## Training YOLO

In [ ]:
model_path = f"./checkpoints/yolo11n.pt"
model = YOLO(model_path)
epochs = 50
batch_size = 64
IMG_SIZE = 640

In [ ]:
# Devi essere in 6D
results = model.train(
        data=f"./datasets/linemod/YOLO/datasets/data.yaml",
        epochs=epochs,
        batch=batch_size,
        device=device,
        imgsz=IMG_SIZE,
        augment=True,
        flipud=0.5,
        fliplr=0.5,
        hsv_h=0.4,
        hsv_s=0.4,
        hsv_v=0.4,
        degrees=120,
        translate=0.1,
        scale=0.5,
        shear=20,
        perspective=0.0001,
        exist_ok=True,
        patience=5, #number of epoch to wait without improvement in validation metrics before early stopping the train. Helps prevent overfitting.
        dropout=0.3
        )

engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation/YOLO/datasets/data.yaml, degrees=120, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.4, hsv_s=0.4, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=./checkpoints/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0001, plots=True, pose

100%|██████████| 755k/755k [00:00<00:00, 14.4MB/s]

Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

YOLO11n summary: 181 layers, 2,592,375 parameters, 2,592,359 gradients, 6.5 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 63.9±135.5 ms, read: 1.0±0.3 MB/s, size: 460.5 KB)


train: Scanning /content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation/YOLO/datasets/train/labels... 11060 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11060/11060 [11:29<00:00, 16.04it/s]


...

      50/50      9.69G     0.5657     0.2314     0.8764         52        640: 100%|██████████| 173/173 [01:00<00:00,  2.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:10<00:00,  1.79it/s]

                   all       2370       2370      0.999          1      0.995      0.876



50 epochs completed in 1.022 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,584,687 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:13<00:00,  1.44it/s]


                   all       2370       2370      0.998          1      0.995      0.867
                    01        186        186      0.995          1      0.995      0.874
                    02        182        182      0.999          1      0.995      0.855
                    04        180        180      0.999          1      0.995      0.865
                    05        180        180      0.999          1      0.995      0.891
                    06        177        177      0.999          1      0.995      0.867
                    08        178        178          1          1      0.995      0.873
                    09        188        188      0.996          1      0.995      0.879
                    10        188        188      0.999          1      0.995       0.88
                    11        183        183      0.995      0.999      0.995      0.804
                    12        185        185      0.999          1      0.995      0.872
                    1

Copy model file to ```checkpoints```

In [ ]:
shutil.copy(f"./datasets/linemod/YOLO/runs/detect/train/weights/best.pt", f"./checkpoints/best.pt")

'/content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation/checkpoints/best.pt'

## Testing YOLO best.pt

In [ ]:
from ultralytics import YOLO
model = YOLO(f"./checkpoints/best.pt")

In [ ]:
results = model.val(
        data=f"./datasets/linemod/YOLO/datasets/data.yaml",
        epochs=epochs,
        batch=batch_size,
        imgsz=IMG_SIZE,
        device=device,
        split="test"
    )

YOLO11n summary (fused): 100 layers, 2,584,687 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.0 ms, read: 1.3±0.3 MB/s, size: 467.6 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation/YOLO/datasets/test/labels... 2370 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2370/2370 [03:07<00:00, 12.65it/s]


val: New cache created: /content/drive/.shortcut-targets-by-id/1clSdIvJJw8QuywtOyjjWKwogqb-DdNZh/6D_pose_estimation/YOLO/datasets/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:14<00:00,  2.62it/s]


                   all       2370       2370      0.999          1      0.995      0.877
                    01        185        185      0.999          1      0.995      0.864
                    02        182        182      0.999          1      0.995      0.866
                    04        180        180      0.999          1      0.995      0.879
                    05        179        179      0.999          1      0.995      0.914
                    06        177        177      0.999          1      0.995      0.882
                    08        178        178      0.999          1      0.995      0.882
                    09        188        188      0.999          1      0.995      0.898
                    10        188        188          1      0.998      0.995        0.9
                    11        183        183          1          1      0.995      0.812
                    12        186        186      0.999          1      0.995      0.868
                    1

In [ ]:
# from ultralytics import solutions, INUTILE
# cropper = solutions.ObjectCropper(show=True, model= "./checkpoints/best.pt", crop_dir = "./datasets/linemod/YOLO/datasets/val/crop/")

Ultralytics Solutions: ✅ {'source': None, 'model': './checkpoints/best.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': './YOLO/datasets/val/crop/', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

WARNING ⚠️ show=True disabled for crop solution, results will be saved in the directory named: ./YOLO/datasets/val/crop/


## PoseEstimation

### Eseguita solo per sicurezza per avere i gt corretti e norm std

In [ ]:
for el in ["01","02","04","05","06","08","09","10","11","12","13","14","15"]:
  shutil.copy(f"./datasets/linemod/YOLO/datasets_cropped/{el}_gt.yml", f"./datasets/linemod/YOLO/datasets_cropped_no_masked/{el}_gt.yml")

In [ ]:
# Directory base
base_dir = './datasets/linemod/DenseFusion/Linemod_preprocessed/data'
subfolders = [f'{i:02d}' for i in range(1, 16)]

# Trasformazione in tensor [C, H, W]
to_tensor = transforms.ToTensor()

# Somme cumulative
sum_rgb = torch.zeros(3)
sum_sq_rgb = torch.zeros(3)
n_pixels = 0

# Loop su tutte le cartelle
for subfolder in tqdm(subfolders, desc="Processing folders"):
    rgb_dir = os.path.join(base_dir, subfolder, 'rgb')
    if not os.path.isdir(rgb_dir):
        continue
    for filename in os.listdir(rgb_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(rgb_dir, filename)
            img = Image.open(path).convert('RGB')
            img_tensor = to_tensor(img)  # shape: [3, H, W]

            # Aggiungi alla somma
            sum_rgb += img_tensor.sum(dim=[1, 2])
            sum_sq_rgb += (img_tensor ** 2).sum(dim=[1, 2])
            n_pixels += img_tensor.shape[1] * img_tensor.shape[2]

# Calcolo finale
mean = sum_rgb / n_pixels
std = torch.sqrt((sum_sq_rgb / n_pixels) - (mean ** 2))

print(f"Mean per channel: {mean}")
print(f"Std per channel: {std}")

### Partire da qui se non è necessario

In [ ]:
class FeatureExtractor(nn.Module):
    """Extract features from images using ResNet specified backbone
        returns a tensor of shape (batch_size, feature_dim)
    """

    def __init__(self, backbone='resnet18', weights= None):
        """Import Resnet architecture pretrained

        Args:
            backbone (str, optional): Name of the Resnet model. Defaults to 'resnet18'.
            weights (str, optional): Pretrained weights of the Resnet
        """
        super(FeatureExtractor, self).__init__()
        self.weights= weights

        if backbone == 'resnet18':
            self.backbone = models.resnet18(weights= "ResNet18_Weights.IMAGENET1K_V1" if self.weights==None else self.weights)
            self.feature_dim = 512

        elif backbone == 'resnet50':
            self.backbone = models.resnet50(weights= "ResNet50_Weights.IMAGENET1K_V2"  if self.weights==None else self.weights)
            self.feature_dim = 2048

        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        for param in self.backbone.parameters():
            param.requires_grad = False

    def forward(self, x):
        features = self.backbone(x)
        features = self.adaptive_pool(features)
        features = features.view(features.size(0), -1)
        return features

    def get_feature_dim(self):
        return self.feature_dim

In [ ]:
class PosePredictorModel(nn.Module):
    """After Resnet, add a second NN to predict translation and rotation
        returns translation and rotation predictions
    """

    def __init__(self, backbone='resnet18', hidden_dim=512):
        """Initialize model and import Resnet backbone

        Args:
            backbone (str, optional): Name of the Resnet architecture to import. Defaults to 'resnet18'. Otherwise resnet50
            hidden_dim (int, optional): Hidden layer dimension. Defaults to 512.
        """
        super(PosePredictorModel, self).__init__()

        # Feature extractor, import resnet architecture
        self.feature_extractor = FeatureExtractor(backbone)
        feature_dim = self.feature_extractor.get_feature_dim()

        # Fully connected layers for the pose prediction
        # self.fc_layers = nn.Sequential(
        #     nn.Linear(feature_dim, hidden_dim//2),
        #     nn.ReLU(),
        #     nn.Dropout(0.3),
        #     nn.Linear(hidden_dim//2, hidden_dim//2),
        #     nn.ReLU(),
        #     nn.Dropout(0.3),
        #     nn.Linear(hidden_dim//2, hidden_dim//4),
        #     nn.ReLU()
        # )

        self.fc_layers = nn.Sequential(
          nn.Linear(feature_dim, hidden_dim),
          nn.BatchNorm1d(hidden_dim),
          nn.ReLU(),
          nn.Dropout(0.3),

          nn.Linear(hidden_dim, hidden_dim),
          nn.BatchNorm1d(hidden_dim),
          nn.ReLU(),
          nn.Dropout(0.3),

          nn.Linear(hidden_dim, hidden_dim // 2),
          nn.BatchNorm1d(hidden_dim // 2),
          nn.ReLU(),
          nn.Dropout(0.3),

          nn.Linear(hidden_dim // 2, hidden_dim // 4),
          nn.BatchNorm1d(hidden_dim // 4),
          nn.ReLU()
        )

        # Output heads separated for translation e rotation
        self.translation_head = nn.Linear(hidden_dim//4, 3) # // 2 per add con matrice di rotazione, // 4 per quaternion
        self.rotation_head = nn.Linear(hidden_dim//4, 4)  # //2, 9 rotation matrix; //4, 4 per quaternion

    def forward(self, x):
        # Extract features
        features = self.feature_extractor(x)

        #  FC layers
        x = self.fc_layers(features)

        # Predict translation and rotation
        translation = self.translation_head(x)
        rotation_flat = self.rotation_head(x)

        # Reshape rotation matrix in 3x3 shape
        batch_size = rotation_flat.size(0)
        rotation_matrix = rotation_flat.view(batch_size, 4) # 3, 3 for rotation matrix, 4 for quaternion

        return translation, rotation_matrix

In [ ]:
class PoseLoss(nn.Module):
    """Loss function combines translation and rotation
        returns a weighted sum of the translation and rotation losses.
    """

    def __init__(self, alpha=1.0, beta=1.0):
        """Initialize alpha and beta values.

        Args:
            alpha (float, optional): Wieght (importance) to give to translation. Defaults to 1.0.
            beta (float, optional): Wieght (importance) to give to rotation. Defaults to 1.0.
        """
        super(PoseLoss, self).__init__()
        self.alpha = alpha  # peso per translation loss
        self.beta = beta    # peso per rotation loss

    def forward(self, pred_trans, pred_rot, gt_trans, gt_rot):
        """Initialize alpha and beta values.

          Args:
              pred_trans (matrix , optional): Predicted translation.
              pred_rot (matrix, optional): Predicted rotation.
              gt_trans (matrix, optional): Ground truth translation.
              gt_rot (matrix, optional): Ground truth rotation.
        """
        # Translation loss (MSE)
        trans_loss = F.mse_loss(pred_trans, gt_trans)

        # Frobenius norm of the differences between matrices
        rot_loss = F.mse_loss(pred_rot, gt_rot.view(-1, 4)) # view(-1,4) per quaternion, view(-1, 3, 3) per rotation
        # rot_loss = self.quaternion_loss(pred_rot, gt_rot.view(-1, 4)) # view(-1,4) per quaternion, view(-1, 3, 3) per rotation

        # Total Loss
        total_loss = self.alpha * trans_loss + self.beta * rot_loss

        return total_loss, trans_loss, rot_loss

    def quaternion_loss(self, pred_q, gt_q):
        pred_q = F.normalize(pred_q, dim=-1)
        gt_q = F.normalize(gt_q, dim=-1)
        dot = torch.sum(pred_q * gt_q, dim=-1).abs()
        return 1 - dot.mean()

In [ ]:
class PoseEstimationTrainer:
    """Trainer class for the Pose Estimation model.
    """

    def __init__(self, model, train_loader, val_loader, device='cuda', config=None, experiment=None):
        """_summary_

        Args:
            model (torch model): Model to be trained
            train_loader (dataloader): train dataloader
            val_loader (dataloader): validation dataloader
            device (str, optional): cuda or cpu. Defaults to 'cuda'.
            config (dict, optional): configuration file. Defaults to None.
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.config = config or {}

        # Loss function e optimizer
        self.criterion = PoseLoss(alpha=1.0, beta=1.0)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
        # self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max= config['num_epochs'], eta_min=1e-6)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

        # Tracking delle metriche
        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        self.step = 0

        # # Log model architecture to wandb
        # if wandb.run is not None:
        #     wandb.watch(self.model, log="all", log_freq=50)

        self.experiment = experiment


    def train_epoch(self):
        """Train the model for one epoch.

        Returns:
            avg_loss (float): average total loss of the epoch
            avg_trans_loss (float): average translation loss of the epoch
            avg_rot_loss (float): average rotation loss of the epoch
        """
        self.model.train()
        total_loss = 0
        total_trans_loss = 0
        total_rot_loss = 0
        num_batches = len(self.train_loader)

        pbar = tqdm(self.train_loader, desc='Training')

        for batch_idx, batch in enumerate(pbar):
            images = batch['cropped_img']
            gt_trans = batch['translation']
            gt_rot = batch['quaternion'] # ['quaternion'] per quaternion ['rotation']

            # Forward pass
            self.optimizer.zero_grad()
            pred_trans, pred_rot = self.model(images)

            # Compute  loss
            loss, trans_loss, rot_loss = self.criterion(pred_trans, pred_rot, gt_trans, gt_rot)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()

            # Aggiorna metriche
            total_loss += loss.item()
            total_trans_loss += trans_loss.item()
            total_rot_loss += rot_loss.item()

            # Log batch metrics to wandb
            if batch_idx %20 ==0:   # per batch_size==1 sarà da cambiare

                self.experiment.log_metrics({
                    "batch_loss": loss.item(),
                    "batch_trans_loss": trans_loss.item(),
                    "batch_rot_loss": rot_loss.item(),
                    "learning_rate": self.optimizer.param_groups[0]['lr'],
                    "step": self.step
                })

            self.step += 1

            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Trans': f'{trans_loss.item():.4f}',
                'Rot': f'{rot_loss.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })

        avg_loss = total_loss / num_batches
        avg_trans_loss = total_trans_loss / num_batches
        avg_rot_loss = total_rot_loss / num_batches

        return avg_loss, avg_trans_loss, avg_rot_loss

    def validate(self):
        """Validate the model on the validation set after each epoch.

        Returns:
            avg_loss (float): average total loss of the epoch
            avg_trans_loss (float): average translation loss of the epoch
            avg_rot_loss (float): average rotation loss of the epoch
        """
        self.model.eval()
        total_loss = 0
        total_trans_loss = 0
        total_rot_loss = 0
        num_batches = len(self.val_loader)

        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc='Validation'):
                images = batch['cropped_img']
                gt_trans = batch['translation']
                gt_rot = batch['quaternion'] # ['quaternion'] per quaternion ['rotation']

                # Forward pass
                pred_trans, pred_rot = self.model(images)

                # Calcola loss
                loss, trans_loss, rot_loss = self.criterion(pred_trans, pred_rot, gt_trans, gt_rot)

                total_loss += loss.item()
                total_trans_loss += trans_loss.item()
                total_rot_loss += rot_loss.item()

        avg_loss = total_loss / num_batches
        avg_trans_loss = total_trans_loss / num_batches
        avg_rot_loss = total_rot_loss / num_batches

        if self.experiment is not None:
            self.experiment.log_metrics({
                "val_loss": avg_loss,
                "val_trans_loss": avg_trans_loss,
                "val_rot_loss": avg_rot_loss,
                "epoch": len(self.train_losses)
            })

        return avg_loss, avg_trans_loss, avg_rot_loss

    def train(self, num_epochs):
        """Train the model for the specified number of epochs.

        Args:
            num_epochs (int): number of epochs to train the model for
        """
        print(f"Starting training for {num_epochs} epochs...")

        if self.experiment is not None:
          with self.experiment.train():
            watch(self.model)

        for epoch in range(num_epochs):
            print(f'\nEpoch {epoch+1}/{num_epochs}')

            # Training
            train_loss, train_trans_loss, train_rot_loss = self.train_epoch()

            # Validation
            val_loss, val_trans_loss, val_rot_loss = self.validate()

            # Scheduler step
            self.scheduler.step()

            # Save metrics
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)

            # Log epoch metrics to wandb
            if self.experiment is not None:
                self.experiment.log_metrics({
                    "epoch": epoch + 1,
                    "train_loss": train_loss,
                    "train_trans_loss": train_trans_loss,
                    "train_rot_loss": train_rot_loss,
                    "val_loss": val_loss,
                    "val_trans_loss": val_trans_loss,
                    "val_rot_loss": val_rot_loss,
                })

            print(f'Train Loss: {train_loss:.4f} (Trans: {train_trans_loss:.4f}, Rot: {train_rot_loss:.4f})')
            print(f'Val Loss: {val_loss:.4f} (Trans: {val_trans_loss:.4f}, Rot: {val_rot_loss:.4f})')

            # Save best model
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss

                os.makedirs(f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/baseline/", exist_ok=True)

                lr = self.optimizer.param_groups[0]['lr']
                batch_size = self.config.get('batch_size', 32)
                best_model_path = (
                    f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/baseline/{self.config['name_saved_file']}_{self.config['backbone']}" # quaternion
                    f"_bs{batch_size}.pth"
                )

                # Save the model
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'config': self.config
                }, best_model_path)

                # Log model to wandb
                if self.experiment is not None:
                    self.experiment.log_metric("best_val_loss", val_loss)

                print(f'Saved best model with val_loss: {val_loss:.4f}')

        print("Training completed!")

    def plot_losses(self):
        """Plot the training and validation losses.
        """
        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(self.train_losses, label='Train Loss', color='blue')
        plt.plot(self.val_losses, label='Validation Loss', color='red')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.grid(True)

        # Log plot to wandb
        if self.experiment is not None:
            self.experiment.log_image(image_data= plt, name= "Plot losses")

        plt.tight_layout()
        plt.show()

In [ ]:
class ADDMetric:
    """ ADD metric to evaluate prediction of the NN comparing it with the CAD model.
        returns the average distance between the predicted pose and the ground truth pose.
    """

    def __init__(self, model, test_loader, models_3D_dir, symmetric_objects=None, device='cuda', experiment=None, config=None):
        """
        Args:
            model (torch model): model to evaluate
            test_loader (dataloader): test loader
            models_3D_dir (str): path to the directory containing 3D models
            symmetric_objects (list, optional): list of object_id that are symmetric. Defaults to None.
            device (str, optional): device to use. Defaults to 'cuda'.
        """
        self.model = model
        self.model.eval()
        self.test_loader = test_loader
        self.models_3D_dir = models_3D_dir
        self.symmetric_objects = symmetric_objects or []
        self.device = device
        self.models_points_dict = self.load_models_points(self.models_3D_dir)
        # read the diameters for each object
        self.objects_info = self.load_objects_models_info()
        self.experiment = experiment
        self.config = config

    def load_models_points(self, models_dir):
        """Load the 3D model points (vertices) for the LINEMOD dataset in a dictionary {class_name: points}.

        Args:
            models_dir (_type_): path of the .ply files

        Returns:
            model_points_dict: dictionary of model points for each object class
        """
        model_points_dict = {}
        class_names = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]

        for obj_id in class_names:
            model_path = os.path.join(models_dir, f'obj_{obj_id}.ply')
            if os.path.exists(model_path):
                try:
                    # Carica il modello 3D
                    mesh = trimesh.load(model_path)

                    # Estrai punti dalla superficie o usa vertices
                    if hasattr(mesh, 'vertices') and mesh.vertices is not None:
                        points = torch.tensor(mesh.vertices/1000.0, dtype=torch.float32).to(device)
                    else:
                        continue

                    # Check for NaN or infinite values
                    if torch.any(torch.isnan(points)) or torch.any(torch.isinf(points)):
                        # Remove NaN/Inf points
                        valid_mask = ~(torch.any(torch.isnan(points), dim=1) | torch.any(torch.isinf(points), dim=1))
                        points = points[valid_mask].to(device)

                    if len(points) == 0:
                        print(f"No valid points found for object {obj_id}")
                        continue
                    model_points_dict[obj_id] = points

                except Exception as e:
                    print(f"Error loading model {model_path}: {e}")
                    continue

        return model_points_dict

    def load_objects_models_info(self):
        with open(f"{self.models_3D_dir}/models_info.yml", 'r') as f:
            return yaml.load(f, Loader=yaml.CLoader)

    def compute_add(self, pred_pose, gt_pose, object_id, threshold=0.1):
        """
        Compute ADD metric for a single prediction

        Args:
            pred_pose (tuple): (translation, rotation_matrix)
            gt_pose (tuple): (translation, rotation_matrix)
            object_id (str): ID of the object
            threshold (float): threshold to consider the prediction correct
        """
        pred_trans, pred_rot = pred_pose
        gt_trans, gt_rot = gt_pose

        # Check for NaN values in poses
        if torch.any(torch.isnan(pred_trans)) or torch.any(torch.isnan(pred_rot)):
            print(f"NaN values found in predicted pose in object {object_id}")
            return torch.nan, False

        if torch.any(torch.isnan(gt_trans)) or torch.any(torch.isnan(gt_rot)):
            print(f"NaN values found in ground truth pose in object {object_id}")
            return torch.nan, False

        try:
        # Convert Torch quaternion [w, x, y, z] -> numpy.quaternion(w, x, y, z)
            pred_quat = np.quaternion(pred_rot[0].item(), pred_rot[1].item(), pred_rot[2].item(), pred_rot[3].item())
            gt_quat = np.quaternion(gt_rot[0].item(), gt_rot[1].item(), gt_rot[2].item(), gt_rot[3].item())

            # Get rotation matrices (3x3 numpy)
            pred_rot = torch.tensor(quaternion.as_rotation_matrix(pred_quat), dtype=torch.float32, device=pred_rot.device)
            gt_rot = torch.tensor(quaternion.as_rotation_matrix(gt_quat), dtype=torch.float32, device=gt_rot.device)

        except Exception as e:
            print(f"Error converting quaternion to rotation matrix for object {object_id}: {e}")
            return torch.nan, False

        # Take 3D model points
        if object_id not in self.models_points_dict:
            print(f"Object with object id {object_id} is not present in models_point_dict")
            return torch.nan, False

        model_points = self.models_points_dict[object_id]  # Shape: (N, 3)

        # Ensure proper shapes
        if pred_rot.shape != (3, 3) or gt_rot.shape != (3, 3):
            print(f"Invalid shape for rotation in object {object_id}")
            return torch.nan, False

        try:
            # Transform points with the predicted pose
            pred_points = torch.matmul(model_points, pred_rot.T) + pred_trans.reshape(1, 3)

            # Transform points with the gt pose
            gt_points = torch.matmul(model_points, gt_rot.T) + gt_trans.reshape(1, 3)

            # Check for NaN in transformed points
            if torch.any(torch.isnan(pred_points)) or torch.any(torch.isnan(gt_points)):
                print(f"NaN values found in transformed points in object {object_id}")
                return torch.nan, False

            # Compute ADD
            if object_id in self.symmetric_objects:
                # For symmetric objects, compute ADD-S
                distances = []
                for pred_point in pred_points:
                    diffs = gt_points - pred_point.reshape(1, 3)
                    point_distances = torch.linalg.norm(diffs, axis=1)
                    min_dist = torch.min(point_distances)
                    distances.append(min_dist)
                distances = torch.tensor(distances)
            else:
                # For non-symmetric objects, compute standard ADD
                distances = torch.linalg.norm(pred_points - gt_points, axis=1)

            avg_distance = torch.mean(distances)

            # Check if the prediction is correct
            is_correct = 1.0 if avg_distance < threshold else 0.0

            return avg_distance, is_correct

        except Exception as e:
            print(f"Errore dirante il calcolo ADD per l'oggetto {object_id}")
            return torch.nan, False

    def evaluate_model_with_add(self):
        """Evaluate the model using the ADD metric."""
        self.model.eval()

        results = {}
        all_distances = []
        all_correct = []
        total_processed = 0
        total_skipped = 0

        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(self.test_loader, desc='Evaluating ADD')):
                images = batch['cropped_img'].to(self.device)
                gt_trans = batch['translation'].to(device)
                gt_rot = batch['quaternion'].to(device)
                object_ids = batch['obj_id'].to(device)

                # Predict
                try:
                    pred_trans, pred_rot = self.model(images)

                except Exception as e:
                    raise ValueError(f"Error during model prediction: {e}")

                # Compute ADD for each object
                for i in range(len(images)):
                    obj_id_int = int(object_ids[i])
                    obj_id = f"{obj_id_int:02d}"

                    if obj_id in self.models_points_dict:
                        pred_pose = (pred_trans[i], pred_rot[i])
                        gt_pose = (gt_trans[i], gt_rot[i])

                        distance, is_correct = self.compute_add(
                            pred_pose, gt_pose, obj_id, threshold=0.1
                        )

                        if not torch.isnan(distance):
                            all_distances.append(distance)
                            all_correct.append(is_correct)
                            total_processed += 1

                            if obj_id not in results:
                                results[obj_id] = {'distances': [], 'correct': []}

                            results[obj_id]['distances'].append(distance)
                            results[obj_id]['correct'].append(is_correct)
                        else:
                            total_skipped += 1
                    else:
                        total_skipped += 1

        if len(all_distances) == 0:
            print("No valid objects found for evaluation.")
            return torch.nan, torch.nan, {}

        # Compute overall metrics
        all_distances = torch.tensor(all_distances)
        all_correct = torch.tensor(all_correct)
        overall_add = torch.mean(all_distances)
        overall_accuracy = torch.mean(all_correct)

        if self.experiment is not None:
            self.experiment.log_metrics({
                "test_add_score": overall_add,
                "test_accuracy": overall_accuracy,
                "total_processed": total_processed,
                "total_skipped": total_skipped
            })

        print(f"\nOverall ADD: {overall_add:.4f}")
        print(f"Overall Accuracy: {overall_accuracy:.4f}")

        # Print per-object results
        print("\nPer-object results:")
        for obj_id, obj_results in results.items():
            obj_add = torch.mean(torch.tensor(obj_results['distances']))
            obj_acc = torch.mean(torch.tensor(obj_results['correct']))
            num_samples = len(obj_results['distances'])
            print(f"Object {obj_id}: ADD={obj_add:.4f}, Acc={obj_acc:.4f}, Samples={num_samples}")

        return overall_add, overall_accuracy, results

In [ ]:
def plotPose(pathImage, translation_gt, rotation_gt, translation_pred, rotation_pred, experiment=None):

    image = cv2.imread(pathImage)
    transparent_image = image.copy()

    rotat_gt = rotation_gt.to(device).float()
    trans_gt = translation_gt.to(device).float()
    rotat_pred = rotation_pred.to(device).float()
    trans_pred = translation_pred.to(device).float()

    image_id = pathImage.split("/")[-1].split(".")[0]
    label = pathImage.split("/")[-3]
    camera_intrinsics = torch.tensor([572.4114, 0.0, 325.2611, 0.0, 573.57043, 242.04899, 0.0, 0.0, 1.0]).reshape(3, 3).to(device)

    meshModel = trimesh.load(f"./Linemod_preprocessed/models/obj_{label}.ply")
    vertices = torch.tensor(meshModel.vertices / 1000, dtype=torch.float32).to(device)
    min_corner = vertices.min(dim=0).values
    max_corner = vertices.max(dim=0).values

    bounding_box_3d = torch.tensor([
        [min_corner[0], min_corner[1], min_corner[2]],
        [max_corner[0], min_corner[1], min_corner[2]],
        [max_corner[0], max_corner[1], min_corner[2]],
        [min_corner[0], max_corner[1], min_corner[2]],
        [min_corner[0], min_corner[1], max_corner[2]],
        [max_corner[0], min_corner[1], max_corner[2]],
        [max_corner[0], max_corner[1], max_corner[2]],
        [min_corner[0], max_corner[1], max_corner[2]],
    ], dtype=torch.float32).to(device)

    if rotat_gt.numel() == 4:
        rotat_gt = torch.tensor(
            quaternion.as_rotation_matrix(np.quaternion(*rotat_gt.cpu().numpy())),
            dtype=torch.float32
        ).to(device)
    else:
        rotat_gt = rotat_gt.reshape(3, 3)

    if rotat_pred.numel() == 4:
        rotat_pred = torch.tensor(
            quaternion.as_rotation_matrix(np.quaternion(*rotat_pred.cpu().numpy())),
            dtype=torch.float32
        ).to(device)
    else:
        rotat_pred = rotat_pred.reshape(3, 3)

    axes_3d = torch.tensor([
        [0, 0, 0],
        [0.15, 0, 0],
        [0, 0.15, 0],
        [0, 0, 0.15]
    ], dtype=torch.float32).to(device)

    axes_cam_gt = (rotat_gt @ axes_3d.T).T + trans_gt
    bounding_box_3d_cam_gt = (rotat_gt @ bounding_box_3d.T).T + trans_gt

    axes_2d_gt = (camera_intrinsics @ axes_cam_gt.T).T
    axes_2d_gt = axes_2d_gt[:, :2] / axes_2d_gt[:, 2:3]
    bounding_box_2d_gt = (camera_intrinsics @ bounding_box_3d_cam_gt.T).T
    bounding_box_2d_gt = (bounding_box_2d_gt[:, :2] / bounding_box_2d_gt[:, 2:3]).int()

    p_gt = [tuple(el.cpu().numpy()) for el in bounding_box_2d_gt]
    edges = [(0,1), (1,2), (2,3), (3,0), (0,4), (1,5), (2,6), (3,7), (4,5), (5,6), (6,7), (7,4)]
    for el in edges:
        cv2.line(image, p_gt[el[0]], p_gt[el[1]], (0,0,255), 5)

    p0_gt = tuple(axes_2d_gt[0].cpu().int().numpy())
    p1_gt = tuple(axes_2d_gt[1].cpu().int().numpy())
    p2_gt = tuple(axes_2d_gt[2].cpu().int().numpy())
    p3_gt = tuple(axes_2d_gt[3].cpu().int().numpy())

    cv2.arrowedLine(image, p0_gt, p1_gt, (0, 0, 255), 2)
    cv2.arrowedLine(image, p0_gt, p2_gt, (0, 255, 0), 2)
    cv2.arrowedLine(image, p0_gt, p3_gt, (255, 0, 0), 2)

    axes_cam_pred = (rotat_pred @ axes_3d.T).T + trans_pred
    bounding_box_3d_cam_pred = (rotat_pred @ bounding_box_3d.T).T + trans_pred

    axes_2d_pred = (camera_intrinsics @ axes_cam_pred.T).T
    axes_2d_pred = axes_2d_pred[:, :2] / axes_2d_pred[:, 2:3]
    bounding_box_2d_pred = (camera_intrinsics @ bounding_box_3d_cam_pred.T).T
    bounding_box_2d_pred = (bounding_box_2d_pred[:, :2] / bounding_box_2d_pred[:, 2:3]).int()

    p_pred = [tuple(el.cpu().numpy()) for el in bounding_box_2d_pred]
    for el in edges:
        cv2.line(image, p_pred[el[0]], p_pred[el[1]], (255, 0, 0), 5)

    p0_pred = tuple(axes_2d_pred[0].cpu().int().numpy())
    p1_pred = tuple(axes_2d_pred[1].cpu().int().numpy())
    p2_pred = tuple(axes_2d_pred[2].cpu().int().numpy())
    p3_pred = tuple(axes_2d_pred[3].cpu().int().numpy())

    cv2.arrowedLine(transparent_image, p0_pred, p1_pred, (0, 0, 255), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p2_pred, (0, 255, 0), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p3_pred, (255, 0, 0), 2)

    overlapImage = cv2.addWeighted(transparent_image, 0.5, image, 1, 0)
    img = cv2.cvtColor(overlapImage, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    if experiment is not None:
      experiment.log_image(image_data=img, name= f"{label}_{image_id}")
    plt.title("Object Pose Estimation (prediction is transparent)")
    plt.show()

### train

In [ ]:
config = {
    "project_name": "baseline_quaternion",
    "experiment_name": "mse_loss_step_optim",
    "batch_size": 32,
    "num_epochs": 25,
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "backbone": "resnet18",
    "hidden_dim": 512,
    "img_size": 224,
    "alpha": 1.0,
    "beta": 1.0,
    "add_threshold": 0.1,
    "symmetric_objects": ["10"],
    "name_saved_file": "mse_loss_step"
}

MODELS_DIR = "/content/datasets/linemod/DenseFusion/Linemod_preprocessed/models"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {DEVICE}")
print(f"Configuration: {config}")

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)


# # --------------------------
# from torch.utils.data import DataLoader, Subset

# # Numero di campioni desiderati nel subset
# subset_size = 10
# subset_indices = list(range(subset_size))

# # Creiamo i subset dei dataset originali
# train_subset = Subset(train_loader.dataset, subset_indices)
# val_subset = Subset(val_loader.dataset, subset_indices)
# test_subset = Subset(test_loader.dataset, subset_indices)

# # Creiamo i nuovi DataLoader a partire dai subset
# train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True)
# val_loader = DataLoader(val_subset, batch_size=config["batch_size"], shuffle=False)
# test_loader = DataLoader(test_subset, batch_size=config["batch_size"], shuffle=False)
# # ----------------------


# Modello
model = PosePredictorModel(
    backbone=config["backbone"],
    hidden_dim=config["hidden_dim"]
).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

experiment = comet_ml.start(
    api_key="qt32gGMiNUQ0MX8Kc1jhRdvlo",
    project_name=config['project_name'],
    experiment_config=comet_ml.ExperimentConfig(
        name=config["experiment_name"],
        parse_args=False)
)

experiment.log_parameters(config)

trainer = PoseEstimationTrainer(model, train_loader, val_loader, device=DEVICE, config=config, experiment=experiment)
trainer.train(num_epochs=config["num_epochs"])

!mkdir -p /content/checkpoints
!cp -r /content/drive/MyDrive/6D_pose_estimation/checkpoints /content


checkpoint = torch.load(f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/baseline/{config['name_saved_file']}_{config['backbone']}_bs{config['batch_size']}.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()


add_metric = ADDMetric(
    model=model,
    test_loader=test_loader,
    models_3D_dir=MODELS_DIR,
    symmetric_objects=config["symmetric_objects"],
    device=DEVICE,
    experiment=experiment,
    config=config
)

print("Evaluating with ADD metric...")
add_score, accuracy, detailed_results = add_metric.evaluate_model_with_add()


print(f"\nFinal Results:\nADD Score: {add_score:.4f}\nAccuracy: {accuracy:.4f}")

test_batch = next(iter(test_loader))

for idx, batch in enumerate(test_loader):
    images = batch['rgb'].to(device)
    gt_trans = batch['translation']
    gt_rot = batch['rotation']
    object_ids = batch['obj_id']
    sample_id = batch["sample_id"]

    with torch.no_grad():
      pred_trans, pred_rot = model(images)
      pred_trans = pred_trans
      pred_rot = pred_rot

      for i in range(len(images)):
        if i == 0:
          img_path = f"./Linemod_preprocessed/data/{sample_id[i][0]:02d}/rgb/{sample_id[i][1]:04d}.png"

          plotPose(img_path, gt_trans[i], gt_rot[i], pred_trans[i], pred_rot[i], experiment)
print(f"Plot salvati su comet_ml in project: {config['project_name']}, experiment: {config['experiment_name']}")

experiment.end()

# Extensions

## PointeNet++

#### trova immagini che non compaiono


In [ ]:
count=0
for el in os.listdir("/content/datasets/linemod/DenseFusion/Linemod_preprocessed/data"):
  for i in os.listdir(f"/content/datasets/linemod/DenseFusion/Linemod_preprocessed/data/{el}/mask"):
    count+=1

print(count)

15783


In [ ]:
print(len(train_dataset)+ len(val_dataset)+len(test_dataset))

15800


In [ ]:
import os

def compare_rgb_mask_in_data(root, extensions={'.png', '.jpg', '.jpeg', '.bmp'}):
    """
    Per ogni classe in `root`, confronta i file tra le cartelle `rgb/` e `mask/`.
    Stampa i file presenti solo in una delle due.

    Args:
        root (str): Percorso della cartella principale (es. "data/")
        extensions (set): Estensioni immagine da considerare
    """
    classes = sorted(os.listdir(root))
    only_rgb= 0
    only_mask=0
    print(classes)

    for class_name in classes:
        class_path = os.path.join(root, class_name)
        rgb_path = os.path.join(class_path, 'rgb')
        mask_path = os.path.join(class_path, 'mask')

        if not os.path.isdir(rgb_path) or not os.path.isdir(mask_path):
            print(f"⚠️  Mancano cartelle rgb o mask in '{class_name}'")
            continue

        rgb_files = {f for f in os.listdir(rgb_path) if os.path.splitext(f)[1].lower() in extensions}
        mask_files = {f for f in os.listdir(mask_path) if os.path.splitext(f)[1].lower() in extensions}

        only_in_rgb = sorted(rgb_files - mask_files)
        only_in_mask = sorted(mask_files - rgb_files)

        if only_in_rgb:
            print(f"\n📁 Classe '{class_name}' — solo in *rgb*:")
            for f in only_in_rgb:
                print(f"  {f}")
                only_rgb+=1

        if only_in_mask:
            print(f"\n📁 Classe '{class_name}' — solo in *mask*:")
            for f in only_in_mask:
                print(f"  {f}")
                only_mask+=1

    print(f"Totale file solo in rgb: {only_rgb}")
    print(f"Totale file solo in mask: {only_mask}")


In [ ]:
compare_rgb_mask_in_data("/content/datasets/linemod/DenseFusion/Linemod_preprocessed/data/")

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '14', '15']

📁 Classe '02' — solo in *mask*:
  1214.png

📁 Classe '15' — solo in *rgb*:
  1225.png
  1226.png
  1227.png
  1228.png
  1229.png
  1230.png
  1231.png
  1232.png
  1233.png
  1234.png
  1235.png
  1236.png
  1237.png
  1238.png
  1239.png
  1240.png
  1241.png
  1242.png
Totale file solo in rgb: 18
Totale file solo in mask: 1


#### CustomDatasetPointCloud

In [61]:
class CustomDataset(Dataset): # used to load and preprocess data
    def __init__(self, dataset_root, split='train', train_ratio=0.7, seed=42):
        """
        Args:
            dataset_root (str): Path to the dataset directory.
            split (str): 'train', 'validation' or 'test'.
            train_ratio (float): Percentage of data used for training (default 70%).
            seed (int): Random seed for reproducibility.
        """
        self.dataset_root = dataset_root
        self.split = split
        self.train_ratio = train_ratio
        self.seed = seed

        # Get list of all samples (folder_id, sample_id)
        self.samples = self.get_all_samples()
        self.camera_intrinsics = torch.tensor([572.4114, 573.57043, 325.2611, 242.04899]).to(device)

        # Check if samples were found
        if not self.samples:
            raise ValueError(f"No samples found in {self.dataset_root}. Check the dataset path and structure.")

        # Split into training and validation+test sets
        labels = [el[0] for el in self.samples]
        self.train_samples, self.val_test_samples = train_test_split(
            self.samples, train_size=self.train_ratio, random_state=self.seed, stratify=labels
        )

        # split validation+test set (by default 30% of the original dataset) into validation and test sets
        labels = [el[0] for el in self.val_test_samples]
        self.val_samples, self.test_samples = train_test_split(self.val_test_samples, train_size=0.5, random_state=self.seed, stratify=labels)

        # Select the appropriate split
        if split == "train":
            self.samples = self.train_samples
        elif split == "validation":
            self.samples = self.val_samples
        else:
            self.samples = self.test_samples

        # Define image transformations: QUESTO PER LA BASELINE
        if self.split == 'train':

            self.transform_img = transforms.Compose([
                               transforms.ToTensor(),  # converte in float32 e normalizza in [0, 1]
                            ])

            self.transform_crop = transforms.Compose([
                                # transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05),
                                # transforms.RandomGrayscale(p=0.1),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.1),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
    ])
        else:
              self.transform_img = transforms.Compose([
                                transforms.ToTensor()
                            ])

              self.transform_crop = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.3348, 0.3165, 0.3105], std=[0.2521, 0.2496, 0.2502])
              ])

        self.ground_truths = {}

        for el in ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]:

          pose_file = os.path.join(self.dataset_root, f"{el}_gt.yml")

          with open(pose_file, 'r') as f:
            pose_data = yaml.load(f, Loader=yaml.CLoader)

          dati = {}
          chiavi_da_estrarre = ['cam_t_m2c', 'cam_R_m2c', 'quaternion', 'obj_bb', 'obj_id']
          dati_estratti = {}

          for key, value in pose_data.items():
            entry = value[0]

            # Estraggo solo le chiavi desiderate
            estratti = {k: entry[k] for k in chiavi_da_estrarre if k in entry}

            # Salvo nel dizionario dei dati estratti
            dati_estratti[key]= estratti
          self.ground_truths[el] = dati_estratti

    def get_samples_id(self):
        return self.samples

    def get_all_samples(self):
        """Retrieve the list of sample indices from all folders, skipping those that already have a mask."""
        samples = []
        for folder_id in range(1, 16):  # Assuming folders are named 01 to 15
            rgb_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", "rgb")
            mask_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", "mask")

            if os.path.exists(rgb_path):
                # Prendi tutti i nomi dei file RGB
                sample_ids = sorted([
                    int(f.split('.')[0]) for f in os.listdir(rgb_path)
                    if f.endswith('.png') and os.path.exists(os.path.join(mask_path, f))
                ])
                samples.extend([(folder_id, sid) for sid in sample_ids])  # Store (folder_id, sample_id)
        return samples


    # def load_config(self, folder_id):
    #     """Load YAML configuration files for camera intrinsics and object info for a specific folder."""
    #     # camera_intrinsics_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", 'info.yml')
    #     objects_info_path = os.path.join(self.dataset_root, 'models', f"models_info.yml")

    #     # with open(camera_intrinsics_path, 'r') as f:
    #         # camera_intrinsics = yaml.load(f, Loader=yaml.FullLoader)
    #     camera_intrinsics = [572.4114, 573.57043, 325.2611, 242.04899]

    #     with open(objects_info_path, 'r') as f:
    #         objects_info = yaml.load(f, Loader=yaml.CLoader)

    #     return camera_intrinsics, objects_info

    #Define here some usefull functions to access the data
    def load_image(self, img_path):
        """Load an RGB image and convert to tensor."""
        img = cv2.imread(img_path, cv2.IMREAD_COLOR_RGB)
        return self.transform_img(img)

    #Define here some usefull functions to access the data
    def load_cropped_image(self, img_path, bbox):
        """Load an RGB image and convert to tensor."""
        img=Image.open(img_path).convert('RGB')
        x,y,w,h = bbox
        cropped_img = img.crop((x,y,x+w,y+h))
        return self.transform_crop(cropped_img)

    def load_depth(self, depth_path):
        """Load a depth image and convert to tensor."""
        return cv2.imread(depth_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

    def load_mask(self, path):
        return cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.uint8)

    def depth_to_pointcloud(self, masked_depth, intrinsics):
        """
        Convert pixel coordinates + depth to 3D metric coordinates using intrinsic parameters.
        Returns x, y, z as Nx3 array.
        """
        fx, fy, cx, cy = intrinsics
        height, width = masked_depth.shape
        valid_mask = masked_depth > 0

        u, v = np.meshgrid(np.arange(width), np.arange(height))
        u_valid = torch.tensor(u[valid_mask]).to(device)
        v_valid = torch.tensor(v[valid_mask]).to(device)
        z_valid = torch.tensor(masked_depth[valid_mask] / 1000.0).to(device)

        # Ensure z_valid is not zero or very small to avoid division by zero
        valid_depth_mask = z_valid > 1e-6
        u_valid = u_valid[valid_depth_mask]
        v_valid = v_valid[valid_depth_mask]
        z_valid = z_valid[valid_depth_mask]

        x_meters = (u_valid - cx) * z_valid / fx
        y_meters = (v_valid - cy) * z_valid / fy
        z_meters = z_valid

        pointcloud = torch.stack([x_meters, y_meters, z_meters], axis=-1).reshape(-1, 3)  # Shape: (N, 3)

        return pointcloud

    def load_6d_pose(self, folder_id, sample_id):
        """Load the 6D pose (translation and rotation) for the object in this sample."""
        pose = self.ground_truths[f"{folder_id:02d}"][int(sample_id)]
        translation = np.array(pose['cam_t_m2c'], dtype=np.float32)/1000.0  # [3] ---> (x,y,z)
        rotation = np.array(pose['cam_R_m2c'], dtype=np.float32).reshape(3, 3)  # [3x3] ---> rotation matrix
        quaternion = np.array(pose['quaternion'], dtype=np.float32)  # [4] ---> quaternion
        bbox_base = np.array(pose['obj_bb'], dtype=np.float32) # [4] ---> x_min, y_min, width, height
        cropped_img = self.load_cropped_image(os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", "rgb", f"{sample_id:04d}.png"), np.array(pose['obj_bb'], dtype=np.float32))
        obj_id = np.array(pose['obj_id'], dtype=np.float32) # [1] ---> label
        return cropped_img, translation, rotation, quaternion, bbox_base, obj_id

    def __len__(self):
        #Return the total number of samples in the selected split.
        return len(self.samples)

    def __getitem__(self, idx):
        #Load a dataset sample.
        folder_id, sample_id = self.samples[idx]
        img_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"rgb/{sample_id:04d}.png")
        depth_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"depth/{sample_id:04d}.png")
        mask_path = os.path.join(self.dataset_root, 'data', f"{folder_id:02d}", f"mask/{sample_id:04d}.png")
        img = self.load_image(img_path)
        depth = self.load_depth(depth_path)
        mask = self.load_mask(mask_path)
        mask_binary = mask != 0
        masked_depth = np.where(mask_binary, depth, 0)

        pointcloud = self.depth_to_pointcloud(masked_depth, self.camera_intrinsics)
        sample_points = fps(pointcloud, None, ratio=802/pointcloud.size(0))[:800]
        pointcloud = pointcloud[sample_points]

        # BISOGNA AGGIUNGERE bbox_YOLO
        cropped_img, translation, rotation, quaternion, bbox_base, obj_id = self.load_6d_pose(folder_id, sample_id)

        #Dictionary with all the data
        return {
            # "rgb": img,
            "cropped_img": cropped_img.to(device),
            # "depth": torch.tensor(depth, dtype=torch.float32),
            "pointcloud": pointcloud.to(device),
            "camera_intrinsics": self.camera_intrinsics,
            "translation": torch.tensor(translation).to(device),
            "rotation": torch.tensor(rotation).to(device),
            "quaternion": torch.tensor(quaternion).to(device),
            "bbox_base": torch.tensor(bbox_base).to(device),
            "obj_id": torch.tensor(obj_id).to(device),
            "sample_id": torch.tensor(self.samples[idx]).to(device)
        }

dataset_root = "/content/datasets/linemod/DenseFusion/Linemod_preprocessed/"

train_dataset = CustomDataset(dataset_root, split='train')
print(f'Training samples: {len(train_dataset)}')

val_dataset = CustomDataset(dataset_root, split='validation')
print(f'Validation samples: {len(val_dataset)}')

test_dataset = CustomDataset(dataset_root, split='test')
print(f'Testing samples: {len(test_dataset)}')

def pointcloud_collate_fn(batch):
  # Trova dimensioni massime per il padding
  max_H = max(item['cropped_img'].shape[1] for item in batch)
  max_W = max(item['cropped_img'].shape[2] for item in batch)

  padded_cropped_imgs = []
  paddings = []

  for item in batch:
    # --- Pad simmetrico immagine ---
    img = item['cropped_img']
    _, H, W = img.shape
    pad_H = max_H - H
    pad_W = max_W - W

    # Calcola padding simmetrico: (left, right, top, bottom)
    pad_left = pad_W // 2
    pad_right = pad_W - pad_left
    pad_top = pad_H // 2
    pad_bottom = pad_H - pad_top

    padding = (pad_left, pad_right, pad_top, pad_bottom)
    padded_img = F.pad(img, padding, mode='replicate')
    padded_cropped_imgs.append(padded_img)
    padding = torch.tensor([pad_left, pad_right, pad_top, pad_bottom])
    paddings.append(padding)

  batch_dict = {
    # "rgb": torch.stack([item['rgb'] for item in batch]).to(device),
    "cropped_img": torch.stack(padded_cropped_imgs).to(device),
    # "depth": torch.stack([item['depth'] for item in batch]).to(device),
    "pointcloud": torch.stack([item['pointcloud'] for item in batch]).to(device),  # Lista di Tensor rimane
    "camera_intrinsics": torch.stack([item['camera_intrinsics'] for item in batch]).to(device),
    "translation": torch.stack([item['translation'] for item in batch]).to(device),
    "rotation": torch.stack([item['rotation'] for item in batch]).to(device),
    "quaternion": torch.stack([item['quaternion'] for item in batch]).to(device),
    "bbox_base": torch.stack([item['bbox_base'] for item in batch]).to(device),
    "obj_id": torch.stack([item['obj_id'] for item in batch]).to(device),
    "sample_id": torch.stack([item['sample_id'] for item in batch]).to(device),
    "paddings":torch.stack(paddings).to(device),
  }

  return batch_dict

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=pointcloud_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=pointcloud_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=pointcloud_collate_fn)

Training samples: 11047
Validation samples: 2367
Testing samples: 2368


#### Architettura extension

##### Architettura

In [38]:
class PointNetLayer(MessagePassing):
    """PointNet layer per estrazione feature geometriche"""
    def __init__(self, in_channels, out_channels, use_xyz=True):
        super().__init__(aggr='max')
        self.use_xyz = use_xyz
        mlp_channels = in_channels + 3 if use_xyz else in_channels
        self.mlp = nn.Sequential(
            nn.Linear(mlp_channels, out_channels),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
            nn.Linear(out_channels, out_channels),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, h, pos, edge_index):
        h = h.view(-1, h.size(-1))
        # h: [B, 1, N, C] oppure [B, N, C] -> Flatten to [B*N, C]
        pos = pos.view(-1, 3)
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self, h_j, pos_j, pos_i):
        geometric_feat = pos_j - pos_i
        if self.use_xyz:
            geometric_feat = torch.cat([h_j, geometric_feat], dim=-1)
        else:
            geometric_feat = h_j
        return self.mlp(geometric_feat)


class SetAbstractionLayer(nn.Module):
    """Set Abstraction per PointNet++"""
    def __init__(self, n_points, radius, n_neighbors, in_channels, out_channels):
        super().__init__()
        self.n_points = n_points
        self.radius = radius
        self.n_neighbors = n_neighbors
        self.pointnet_layer = PointNetLayer(in_channels, out_channels)

    def forward(self, pos, h=None, batch_indices=None):
        if h is None:
          h = pos

        # Sampling with proper batch handling
        if self.n_points is not None:
            pos = pos.view(-1, 3)  # -> [B, N, 3] -> [B*N, 3]
            # Campiona n_points usando FPS
            centroids_idx = fps(pos, batch_indices, ratio=self.n_points / pos.size(0))
            centroids = pos[centroids_idx] # [N_centroids, 3]
            centroids_batch = batch_indices[centroids_idx]# [N_centroids]
        else:
            centroids_idx = torch.arange(pos.size(0), device=pos.device)
            centroids = pos
            centroids_batch = batch_indices

        edge_index = radius(pos, centroids, r=self.radius,
                               batch_x=batch_indices, batch_y=centroids_batch,
                               max_num_neighbors=self.n_neighbors) # [2, E]
        # PointNet
        aggregated_h = self.pointnet_layer(h, pos, edge_index) # [N, out_channels]
        # Aggregate per centroids
        new_h = torch.zeros(centroids.size(0), aggregated_h.size(1),
                           device=aggregated_h.device, dtype=aggregated_h.dtype) # [N_centroids, out_channels]

        for i, centroid_idx in enumerate(centroids_idx):
            mask = edge_index[1] == i
            if mask.any():
                neighbor_feats = aggregated_h[edge_index[0][mask]]
                new_h[i] = neighbor_feats.max(dim=0)[0]

        return centroids, new_h, centroids_batch, edge_index # [N_centroids, 3] [N_centroids, out_channels] [N_centroids], [2, E]


class GeometricFeatureExtractor(nn.Module):
    """
    Estrattore di feature geometriche da point cloud
    Produce embeddings multi-scala per la fusion
    """
    def __init__(self, feature_dims=[64, 128, 256]):
        super().__init__()
        self.feature_dims = feature_dims

        # Multi-scale feature extraction
        self.sa1 = SetAbstractionLayer(
            n_points=800, radius=0.05, n_neighbors=32,
            in_channels=3, out_channels=feature_dims[0]
        )
        self.sa2 = SetAbstractionLayer(
            n_points=200, radius=0.1, n_neighbors=64,
            in_channels=feature_dims[0], out_channels=feature_dims[1]
        )
        self.sa3 = SetAbstractionLayer(
            n_points=50, radius=0.2, n_neighbors=64,
            in_channels=feature_dims[1], out_channels=feature_dims[2]
        )

        # Feature projection layers per diversi livelli
        self.feat_proj1 = nn.Sequential(
            nn.Linear(feature_dims[0], feature_dims[0]),
            nn.BatchNorm1d(feature_dims[0]),
            nn.ReLU()
        )
        self.feat_proj2 = nn.Sequential(
            nn.Linear(feature_dims[1], feature_dims[1]),
            nn.BatchNorm1d(feature_dims[1]),
            nn.ReLU()
        )
        self.feat_proj3 = nn.Sequential(
            nn.Linear(feature_dims[2], feature_dims[2]),
            nn.BatchNorm1d(feature_dims[2]),
            nn.ReLU()
        )

    def upsampling_features(self, pos, h, edge_index):
      # pos [B*N, 3], h [M, C], edge:index [2, E]

      # edge_index[0] -> idx nei punti originali (pos)
      # edge_index[1] -> idx nei centroidi

      # Crea un tensore vuoto per i punti originali
      all_feats = torch.zeros(pos.shape[0], h.shape[1], device=pos.device) # [B*N, C]

      # Conta quanti centroidi sono associati a ciascun punto
      counts = torch.zeros(pos.shape[0], 1, device=pos.device) # [B*N, 1]

      # Somma le feature di ciascun centroide a tutti i punti vicini
      all_feats.index_add_(0, edge_index[0], h[edge_index[1]])
      counts.index_add_(0, edge_index[0], torch.ones_like(h[edge_index[1]][:, :1]))

      # Media (evita divisione per zero)
      counts = counts.clamp(min=1)
      all_feats = all_feats / counts
      return all_feats

    def forward(self, batch):
        """
        Args:
            batch: dict containing 'pointcloud': [B, N, 3] point cloud coordinates
        Returns:
            multi_scale_features: dict con features a diverse risoluzioni
        """
        pointcloud = batch['pointcloud']  # [B, N, 3]
        B, N, _ = pointcloud.shape

        # Flatten pointcloud and create batch indices
        pos = pointcloud.view(-1, 3)  # [B*N, 3]
        batch_indices = torch.arange(B, device=pos.device).repeat_interleave(N)  # [B*N]

        # Extract multi-scale features with proper batch tracking
        # [B*800, 3] [B*800, 64] [B*800] output shapes
        pos1, h1, batch1, edge_index1 = self.sa1(pos, batch=batch_indices)      # [B*800, 64]
        # [B*200, 3] [B*200, 128] [B*200] output shapes
        pos2, h2, batch2, edge_index2 = self.sa2(pos1, h1, batch1)             # [B*200, 128]
        # [B*50, 3] [B*50, 256] [B*50] output shapes
        pos3, h3, batch3, edge_index3 = self.sa3(pos2, h2, batch2)             # [B*50, 256]

        h1 = self.feat_proj1(h1)
        h2 = self.feat_proj2(h2)
        h3 = self.feat_proj3(h3)

        h2 = self.upsampling_features(pos, h2, edge_index2)  # [B*800, 128]
        h3 = self.upsampling_features(pos2, h3, edge_index3) # [B*200, 256]
        h3 = self.upsampling_features(pos, h3, edge_index2) # [B*800, 256]

        return {
            'level1': {'pos': pos1, 'features': h1, 'batch': batch1},  # Fine details h1 [B*800, 64]
            'level2': {'pos': pos2, 'features': h2, 'batch': batch2},  # Medium details h2 [B*800, 128]
            'level3': {'pos': pos3, 'features': h3, 'batch': batch3},  # Coarse details h3 [B*800, 256]
        }


class PointProjector(nn.Module):
    """
    Proietta i punti 3D nello spazio immagine usando parametri intrinseci fissi
    Assume che i punti siano già nel sistema di coordinate della camera
    """
    def __init__(self, fx=572.4114, fy=573.57043, cx=325.2611, cy=242.04899):
        super().__init__()
        # Fixed camera intrinsics matrix
        self.register_buffer('camera_intrinsics', torch.tensor([
            [fx, 0.0, cx],
            [0.0, fy, cy],
            [0.0, 0.0, 1.0]
        ]))

    def forward(self, points_3d, bbox_base, padding):
        """
        Args:
            points_3d: [N, 3] coordinate 3D dei punti nel sistema camera
            bbox_base: [4] (x_min, y_min, crop_width, crop_height)
            padding: [4] (pad_left, pad_right, pad_top, pad_bottom)
        Returns:
            pixel_coords: [N, 2] coordinate pixel (u, v) rispetto all'immagine croppata e paddata
            valid_mask: [N] mask per punti validi (dentro l'immagine croppata e paddata)
        """
        device = points_3d.device

        # Estrai parametri bbox e padding
        x_min, y_min, crop_width, crop_height = bbox_base.int()
        pad_left, pad_right, pad_top, pad_bottom = padding.int()

        # 1. Project to original image plane (640x480)
        points_2d_homo = torch.matmul(points_3d, self.camera_intrinsics.T)
        pixel_coords_original = points_2d_homo[:, :2] / (points_2d_homo[:, 2:3] + 1e-8)

        # 2. Transform to cropped image coordinates
        pixel_coords_crop = torch.zeros_like(pixel_coords_original)
        pixel_coords_crop[:, 0] = pixel_coords_original[:, 0] - x_min  # u coordinate
        pixel_coords_crop[:, 1] = pixel_coords_original[:, 1] - y_min  # v coordinate

        # 3. Transform to padded image coordinates
        pixel_coords_padded = torch.zeros_like(pixel_coords_crop)
        pixel_coords_padded[:, 0] = pixel_coords_crop[:, 0] + pad_left
        pixel_coords_padded[:, 1] = pixel_coords_crop[:, 1] + pad_top

        # 4. Valid mask checks
        # Check depth
        valid_depth = points_3d[:, 2] > 0.1  # min depth

        # Check if points are within original crop area
        valid_in_original_crop = (
            (pixel_coords_original[:, 0] >= x_min) &
            (pixel_coords_original[:, 0] < x_min + crop_width) &
            (pixel_coords_original[:, 1] >= y_min) &
            (pixel_coords_original[:, 1] < y_min + crop_height)
        )

        # Check if points are within padded image bounds
        padded_width = crop_width + pad_left + pad_right
        padded_height = crop_height + pad_top + pad_bottom

        valid_in_padded = (
            (pixel_coords_padded[:, 0] >= 0) &
            (pixel_coords_padded[:, 0] < padded_width) &
            (pixel_coords_padded[:, 1] >= 0) &
            (pixel_coords_padded[:, 1] < padded_height)
        )

        valid_mask = valid_depth & valid_in_original_crop & valid_in_padded

        return pixel_coords_padded, valid_mask


class PixelWiseFusionNetwork(nn.Module):
    """
    Network principale per fusion pixel-wise di features geometriche e visive
    """
    def __init__(self,
                 geometric_dims=[64, 128, 256],
                 image_backbone='resnet18',
                 fx=572.4114, fy=573.57043, cx=325.2611, cy=242.04899):
        super().__init__()

        self.geometric_dims = geometric_dims
        self.sample_img_features_dim = 256

        # Geometric feature extractor
        self.geometric_extractor = GeometricFeatureExtractor(geometric_dims)

        # Point projector with fixed intrinsics
        self.point_projector = PointProjector(fx, fy, cx, cy)

        # Image feature extractor (CNN backbone)
        if image_backbone == 'resnet18':
            self.backbone = models.resnet18(weights="ResNet18_Weights.IMAGENET1K_V1")
            self.image_encoder = nn.Sequential(*list(self.backbone.children())[:-2])
            image_feat_dim = 512
        else:
            raise NotImplementedError(f"Backbone {image_backbone} not implemented")

        for param in self.backbone.parameters():
            param.requires_grad = False

        self.features_reduction = nn.Sequential(
            nn.Conv2d(image_feat_dim, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, self.sample_img_features_dim, 1),
            nn.BatchNorm2d(self.sample_img_features_dim),
            nn.ReLU(inplace=True)
            )

    def forward(self, batch):
        """
        Args:
            batch: dictionary containing:
                - cropped_img: [B, 3, H, W] immagini RGB croppate e paddate
                - pointcloud: [B, N, 3] coordinate 3D
                - bbox_base: [B, 4] bbox parameters
                - paddings: [B, 4] padding parameters
        Returns:
            fused_features: [B, fusion_dim, H, W] feature fuse pixel-wise
        """
        B, _, H, W = batch['cropped_img'].shape
        image = batch['cropped_img']

        # Extract geometric features for all batch items
        geometric_features_dict = self.geometric_extractor(batch)

        # Extract image features
        image_features = self.image_encoder(image)  # [B, 512, H_pad/32, W_pad/32]
        image_features = self.features_reduction(image_features)  # [B, 256, H_pad/32, W_pad/32]
        image_features = F.interpolate(image_features, size=(H, W), mode='bilinear', align_corners=False) # [B, 256, H_pad, W_pad]

        # Initialize fusion feature maps for each level
        fusion_maps = []

        geometric_features = torch.cat([level_data['features'] for level_data in geometric_features_dict.values()], dim=1) # [B*N, 448]

        level_data = geometric_features_dict['level1']
        points_3d = level_data['pos']  # All points for all batches [B*N, 3]
        points_batch = level_data['batch']  # Batch assignment for each point [B*N]

        # Process each batch separately
        for b in range(B):
            # Get points belonging to this batch
            batch_mask = points_batch == b

            batch_points = points_3d[batch_mask]  # [N_b, 3]
            batch_geom_feats = geometric_features[batch_mask]  # [N_b, 448]
            bbox_base = batch['bbox_base'][b]  # [4]
            padding = batch['paddings'][b]  # [4]

            # Project points to padded image coordinates
            pixel_coords, valid_mask = self.point_projector(batch_points, bbox_base, padding)

            # Keep only valid points and features
            valid_points = batch_points[valid_mask] # [N_valid, 3]
            valid_geom_feats = batch_geom_feats[valid_mask] # [N_valid, feat_geom]
            valid_pixels = pixel_coords[valid_mask] # [N_valid, 2]

            if len(valid_points) == 0:
                print(f"[DEBUG] No valid points for batch {b} at level {level_name}")
                continue


                #########DA QUIIII

                # Map geometric features to pixels
                u_coords = torch.clamp(valid_pixels[:, 0].long(), 0, W-1)
                v_coords = torch.clamp(valid_pixels[:, 1].long(), 0, H-1)

                # Simple assignment (last point wins if multiple points map to same pixel)
                for i in range(len(valid_geom_feats)):
                    u, v = u_coords[i], v_coords[i]
                    pixel_geom_features[:, v, u] = valid_geom_feats[i]

                # Prepare features for fusion
                # Geometric features: [geom_dim, H, W] -> [H*W, geom_dim]
                pixel_geom_flat = pixel_geom_features.permute(1, 2, 0).reshape(H*W, self.geometric_dims[level_idx])

                # Image features for this batch: [img_dim, H, W] -> [H*W, img_dim]
                image_feat_flat = image_features[b].permute(1, 2, 0).reshape(H*W, image_features.size(1))

                # Concatenate geometric and image features
                combined_feats = torch.cat([pixel_geom_flat, image_feat_flat], dim=1)  # [H*W, geom_dim + img_dim]

                # Apply fusion layer
                fused_feats = self.fusion_layers[level_name](combined_feats)  # [H*W, fusion_dim]

                # Reshape back to spatial dimensions
                level_fusion_features[b] = fused_feats.reshape(H, W, self.fusion_dim).permute(2, 0, 1)  # [fusion_dim, H, W]

            fusion_maps.append(level_fusion_features)

        # Aggregate multi-level features
        multi_level_features = torch.cat(fusion_maps, dim=1)  # [B, fusion_dim*3, H, W]
        final_features = self.feature_aggregator(multi_level_features)

        if self.output_classes is not None:
            output = self.output_head(final_features)
            return final_features, output

        return final_features

class PoseEstimationPipeline(nn.Module):
    """
    Pipeline per stima di posa dense seguendo il paper DenseFusion:
    - Feature pixel-wise -> Global features via MLP + AvgPool
    - Concatenazione global + pixel-wise features
    - Predizione per-pixel di posa + confidenza
    - Argmax per selezione finale
    """
    def __init__(self,
                 geometric_dims=[64, 128, 256],
                 fusion_dim=256,
                 fx=572.4114, fy=573.57043, cx=325.2611, cy=242.04899,
                 img_width=640, img_height=480):
        super().__init__()

        # Fusion network (senza output_classes)
        self.fusion_network = PixelWiseFusionNetwork(
            geometric_dims=geometric_dims,
            fusion_dim=fusion_dim,
            output_classes=None,  # Non serve per pose estimation
            fx=fx, fy=fy, cx=cx, cy=cy,
            img_width=img_width, img_height=img_height
        )

        # MLP per ottenere global features dalle feature pixel-wise
        self.global_feature_mlp = nn.Sequential(
            nn.Conv2d(fusion_dim, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        # Global pooling per ridurre le dimensioni spaziali
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        # MLP finale per predizioni per-pixel
        # Input: fusion_dim (pixel features) + 128 (global features)
        self.final_mlp_quat = nn.Sequential(
            nn.Conv2d(fusion_dim + 128, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.final_mlp_rot = nn.Sequential(
            nn.Conv2d(fusion_dim + 128, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.final_mlp_conf = nn.Sequential(
            nn.Conv2d(fusion_dim + 128, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        # Head per rotazione (quaternion per ogni pixel)
        self.rotation_head = nn.Conv2d(128, 4, 1)  # [B, 4, H, W]

        # Head per traslazione (per ogni pixel)
        self.translation_head = nn.Conv2d(128, 3, 1)  # [B, 3, H, W]

        # Head per confidenza (per ogni pixel)
        self.confidence_head = nn.Conv2d(128, 1, 1)  # [B, 1, H, W]

    def forward(self, batch_data):
        """
        Args:
            batch_data: dict contenente point_cloud, image, batch
        Returns:
            rotation_quaternion: [B, 4] quaternion finale (dopo argmax)
            translation: [B, 3] traslazione finale (dopo argmax)
            pixel_predictions: dict con tutte le predizioni pixel-wise per debugging/loss
        """
        B = batch_data['cropped_img'].shape[0]

        # 1. Estrai feature fuse pixel-wise
        pixel_features = self.fusion_network(batch_data)  # [B, fusion_dim, H, W]
        H, W = pixel_features.shape[2], pixel_features.shape[3]

        # 2. Calcola global features
        global_feat_intermediate = self.global_feature_mlp(pixel_features)  # [B, 128, H, W]
        global_features = self.avg_pool(global_feat_intermediate)  # [B, 128, 1, 1]

        # 3. Espandi global features per ogni pixel
        global_features_expanded = global_features.expand(-1, -1, H, W)  # [B, 128, H, W]

        # 4. Concatena pixel features + global features
        combined_features = torch.cat([pixel_features, global_features_expanded], dim=1)  # [B, fusion_dim+128, H, W]

        # 5. MLP finale
        # final_features = self.final_mlp(combined_features)  # [B, 128, H, W]

        # 6. Predizioni per ogni pixel
        pixel_rotations = self.rotation_head(self.final_mlp_quat(combined_features))  # [B, 4, H, W]
        pixel_translations = self.translation_head(self.final_mlp_rot(combined_features))  # [B, 3, H, W]
        pixel_confidences = self.confidence_head(self.final_mlp_conf(combined_features))  # [B, 1, H, W]

        # 7. Applica sigmoid alla confidenza per normalizzarla in [0,1]
        pixel_confidences = torch.sigmoid(pixel_confidences)

        # 8. Normalizza i quaternion per ogni pixel
        pixel_rotations_norm = F.normalize(pixel_rotations, p=2, dim=1)  # [B, 4, H, W]

        # 9. Argmax sulla confidenza per selezionare il pixel migliore
        confidence_flat = pixel_confidences.view(B, -1)  # [B, H*W]
        best_pixel_indices = torch.argmax(confidence_flat, dim=1)  # [B]

        # 10. Estrai le predizioni finali usando gli indici del best pixel
        final_rotations = torch.zeros(B, 4, device=pixel_rotations.device)
        final_translations = torch.zeros(B, 3, device=pixel_translations.device)
        final_confidences = torch.zeros(B, device=pixel_confidences.device)

        for b in range(B):
            best_idx = best_pixel_indices[b]
            h_idx = best_idx // W
            w_idx = best_idx % W

            final_rotations[b] = pixel_rotations_norm[b, :, h_idx, w_idx]
            final_translations[b] = pixel_translations[b, :, h_idx, w_idx]
            final_confidences[b] = pixel_confidences[b, 0, h_idx, w_idx]

        # Dizionario con tutte le predizioni pixel-wise (utile per loss e debugging)
        pixel_predictions = {
            'rotations': pixel_rotations_norm,  # [B, 4, H, W]
            'translations': pixel_translations,  # [B, 3, H, W]
            'confidences': pixel_confidences,   # [B, 1, H, W]
            'best_pixel_indices': best_pixel_indices,  # [B]
            'best_confidences': final_confidences  # [B]
        }

        return final_rotations, final_translations, pixel_predictions

    def get_pose_at_pixel(self, batch_data, pixel_coords):
        """
        Utility function per ottenere la posa predetta a coordinate pixel specifiche

        Args:
            batch_data: input batch
            pixel_coords: [B, 2] coordinate (u, v) per ogni batch item
        Returns:
            rotations: [B, 4] quaternion alle coordinate specificate
            translations: [B, 3] traslazioni alle coordinate specificate
        """
        with torch.no_grad():
            B = batch_data['cropped_img'].shape[0]

            # Forward per ottenere tutte le predizioni pixel-wise
            _, _, pixel_predictions = self.forward(batch_data)

            pixel_rotations = pixel_predictions['rotations']  # [B, 4, H, W]
            pixel_translations = pixel_predictions['translations']  # [B, 3, H, W]

            # Estrai predizioni alle coordinate specificate
            rotations = torch.zeros(B, 4, device=pixel_rotations.device)
            translations = torch.zeros(B, 3, device=pixel_translations.device)

            for b in range(B):
                u, v = pixel_coords[b]
                u, v = int(u.clamp(0, pixel_rotations.shape[3]-1)), int(v.clamp(0, pixel_rotations.shape[2]-1))
                rotations[b] = pixel_rotations[b, :, v, u]
                translations[b] = pixel_translations[b, :, v, u]

            return rotations, translations

In [19]:
class ADDMetric:
    """ ADD metric to evaluate prediction of the NN comparing it with the CAD model.
        returns the average distance between the predicted pose and the ground truth pose.
    """

    def __init__(self, model, test_loader, models_3D_dir, symmetric_objects=None, device='cuda', experiment=None, config=None):
        """
        Args:
            model (torch model): model to evaluate
            test_loader (dataloader): test loader
            models_3D_dir (str): path to the directory containing 3D models
            symmetric_objects (list, optional): list of object_id that are symmetric. Defaults to None.
            device (str, optional): device to use. Defaults to 'cuda'.
        """
        self.model = model
        self.model.eval()
        self.test_loader = test_loader
        self.models_3D_dir = models_3D_dir
        self.symmetric_objects = symmetric_objects or []
        self.device = device
        self.models_points_dict = self.load_models_points(self.models_3D_dir)
        self.dict_diameters = self.load_diameters()
        self.experiment = experiment
        self.config = config


    def load_diameters(self):
        dict_diameters = {}
        with open('/content/datasets/linemod/DenseFusion/Linemod_preprocessed/models/models_info.yml', 'r') as f:
            fl = yaml.load(f, Loader=yaml.CLoader)

        for k,v in fl.items():
            dict_diameters[f'{k:02d}'] = v['diameter']/1000.0
        return dict_diameters


    def load_models_points(self, models_dir):
        """Load the 3D model points (vertices) for the LINEMOD dataset in a dictionary {class_name: points}.

        Args:
            models_dir (_type_): path of the .ply files

        Returns:
            model_points_dict: dictionary of model points for each object class
        """
        model_points_dict = {}
        class_names = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]

        for obj_id in class_names:
            model_path = os.path.join(models_dir, f'obj_{obj_id}.ply')
            if os.path.exists(model_path):
                try:
                    # Carica il modello 3D
                    mesh = trimesh.load(model_path)

                    # Estrai punti dalla superficie o usa vertices
                    if hasattr(mesh, 'vertices') and mesh.vertices is not None:
                        points = torch.tensor(mesh.vertices/1000.0, dtype=torch.float32).to(device)
                    else:
                        continue

                    # Check for NaN or infinite values
                    if torch.any(torch.isnan(points)) or torch.any(torch.isinf(points)):
                        # Remove NaN/Inf points
                        valid_mask = ~(torch.any(torch.isnan(points), dim=1) | torch.any(torch.isinf(points), dim=1))
                        points = points[valid_mask].to(device)

                    if len(points) == 0:
                        print(f"No valid points found for object {obj_id}")
                        continue
                    model_points_dict[obj_id] = points

                except Exception as e:
                    print(f"Error loading model {model_path}: {e}")
                    continue

        return model_points_dict

    def compute_add(self, pred_pose, gt_pose, object_id, threshold=0.1):
        """
        Compute ADD metric for a single prediction

        Args:
            pred_pose (tuple): (translation, rotation_matrix)
            gt_pose (tuple): (translation, rotation_matrix)
            object_id (str): ID of the object
            threshold (float): threshold to consider the prediction correct
        """
        pred_trans, pred_rot = pred_pose
        gt_trans, gt_rot = gt_pose

        # Check for NaN values in poses
        if torch.any(torch.isnan(pred_trans)) or torch.any(torch.isnan(pred_rot)):
            print(f"NaN values found in predicted pose in object {object_id}")
            return torch.nan, False

        if torch.any(torch.isnan(gt_trans)) or torch.any(torch.isnan(gt_rot)):
            print(f"NaN values found in ground truth pose in object {object_id}")
            return torch.nan, False

        try:
        # Convert Torch quaternion [x, y, z, w] -> numpy.quaternion(w, x, y, z)
            pred_quat = np.quaternion(pred_rot[3].item(), pred_rot[0].item(), pred_rot[1].item(), pred_rot[2].item())
            gt_quat = np.quaternion(gt_rot[3].item(), gt_rot[0].item(), gt_rot[1].item(), gt_rot[2].item())

            # Get rotation matrices (3x3 numpy)
            pred_rot = torch.tensor(quaternion.as_rotation_matrix(pred_quat), dtype=torch.float32, device=pred_rot.device)
            gt_rot = torch.tensor(quaternion.as_rotation_matrix(gt_quat), dtype=torch.float32, device=gt_rot.device)

        except Exception as e:
            print(f"Error converting quaternion to rotation matrix for object {object_id}: {e}")
            return torch.nan, False

        # Take 3D model points
        if object_id not in self.models_points_dict:
            print(f"Object with object id {object_id} is not present in models_point_dict")
            return torch.nan, False

        model_points = self.models_points_dict[object_id]  # Shape: (N, 3)

        # Ensure proper shapes
        if pred_rot.shape != (3, 3) or gt_rot.shape != (3, 3):
            print(f"Invalid shape for rotation in object {object_id}")
            return torch.nan, False

        try:
            # Transform points with the predicted pose
            pred_points = torch.matmul(model_points, pred_rot.T) + pred_trans.reshape(1, 3)

            # Transform points with the gt pose
            gt_points = torch.matmul(model_points, gt_rot.T) + gt_trans.reshape(1, 3)

            # Check for NaN in transformed points
            if torch.any(torch.isnan(pred_points)) or torch.any(torch.isnan(gt_points)):
                print(f"NaN values found in transformed points in object {object_id}")
                return torch.nan, False

            # Compute ADD
            if object_id in self.symmetric_objects:
                # For symmetric objects, compute ADD-S
                distances = []
                for pred_point in pred_points:
                    diffs = gt_points - pred_point.reshape(1, 3)
                    point_distances = torch.linalg.norm(diffs, axis=1)
                    min_dist = torch.min(point_distances)
                    distances.append(min_dist)
                distances = torch.tensor(distances)
            else:
                # For non-symmetric objects, compute standard ADD
                distances = torch.linalg.norm(pred_points - gt_points, axis=1)

            avg_distance = torch.mean(distances)

            # Check if the prediction is correct
            is_correct = 1.0 if avg_distance < threshold else 0.0

            return avg_distance, is_correct

        except Exception as e:
            print(f"Errore dirante il calcolo ADD per l'oggetto {object_id}")
            return torch.nan, False

    def evaluate_model_with_add(self):
        """Evaluate the model using the ADD metric."""
        self.model.eval()

        results = {}
        all_distances = []
        all_correct = []
        total_processed = 0
        total_skipped = 0

        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(self.test_loader, desc='Evaluating ADD')):
                images = batch['cropped_img'].to(self.device)
                gt_trans = batch['translation'].to(device)
                gt_rot = batch['quaternion'].to(device)
                object_ids = batch['obj_id'].to(device)

                # Predict
                try:
                    pred_rot, pred_trans, _ = self.model(batch)

                except Exception as e:
                    raise ValueError(f"Error during model prediction: {e}")

                # Compute ADD for each object

                for i in range(len(images)):
                    obj_id_int = int(object_ids[i])
                    obj_id = f"{obj_id_int:02d}"

                    if obj_id in self.models_points_dict:
                        pred_pose = (pred_trans[i], pred_rot[i])
                        gt_pose = (gt_trans[i], gt_rot[i])

                        distance, is_correct = self.compute_add(
                            pred_pose, gt_pose, obj_id, threshold=0.1*self.dict_diameters[obj_id]
                        )

                        if not torch.isnan(distance):
                            all_distances.append(distance)
                            all_correct.append(is_correct)
                            total_processed += 1

                            if obj_id not in results:
                                results[obj_id] = {'distances': [], 'correct': []}

                            results[obj_id]['distances'].append(distance)
                            results[obj_id]['correct'].append(is_correct)
                        else:
                            total_skipped += 1
                    else:
                        total_skipped += 1

        if len(all_distances) == 0:
            print("No valid objects found for evaluation.")
            return torch.nan, torch.nan, {}

        # Compute overall metrics
        all_distances = torch.tensor(all_distances)
        all_correct = torch.tensor(all_correct)
        overall_add = torch.mean(all_distances)
        overall_accuracy = torch.mean(all_correct)

        if self.experiment is not None:
            self.experiment.log_metrics({
                "test_add_score": overall_add,
                "test_accuracy": overall_accuracy,
                "total_processed": total_processed,
                "total_skipped": total_skipped
            })

        print(f"\nOverall ADD: {overall_add:.4f}")
        print(f"Overall Accuracy: {overall_accuracy:.4f}")

        # Print per-object results
        print("\nPer-object results:")
        for obj_id, obj_results in results.items():
            obj_add = torch.mean(torch.tensor(obj_results['distances']))
            obj_acc = torch.mean(torch.tensor(obj_results['correct']))
            num_samples = len(obj_results['distances'])
            print(f"Object {obj_id}: ADD={obj_add:.4f}, Acc={obj_acc:.4f}, Samples={num_samples}")

        return overall_add, overall_accuracy, results

In [74]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from tqdm import tqdm  # per visualizzare il progresso
# Funzione per invertire la normalizzazione
def denormalize(tensor, mean, std):
    """
    Inverte la normalizzazione di un'immagine tensorizzata [C, H, W]
    """
    mean = torch.tensor(mean).view(-1, 1, 1).to(tensor.device)
    std = torch.tensor(std).view(-1, 1, 1).to(tensor.device)
    return tensor * std + mean

# Parametri della normalizzazione usata durante il training
mean = [0.3348, 0.3165, 0.3105]
std = [0.2521, 0.2496, 0.2502]

# Assicurati di avere istanziato il PointProjector con la camera intrinsics
projector = PointProjector().to(device)

for batch_idx, batch in enumerate(tqdm(train_loader, desc="Processing batches")):
    pointclouds = batch["pointcloud"]        # [B, N, 3]
    bbox_bases = batch["bbox_base"]          # [B, 4]
    paddings = batch["paddings"]             # [B, 4]
    cropped_imgs = batch["cropped_img"]      # [B, 3, H, W]

    B = pointclouds.shape[0]

    for i in range(B):
        points_3d = pointclouds[i]            # [N, 3]
        bbox_base = bbox_bases[i]             # [4]
        padding = paddings[i]                 # [4]
        cropped_img = cropped_imgs[i]         # [3, H, W]

        # Proietta i punti nel piano immagine 2D
        pixel_coords, valid_mask = projector(points_3d, bbox_base, padding)

        # Logging base
        n_valid = valid_mask.sum().item()
        if n_valid != 800:
            print(f"[Batch {batch_idx}, Sample {i}] ⚠️ Valid points: {n_valid}/800")

        # (Opzionale) Visualizza risultati per debugging
        visualize = False
        if visualize:
            img_denorm = denormalize(cropped_img, mean, std)
            img_np = img_denorm.permute(1, 2, 0).cpu().numpy()
            img_np = np.clip(img_np, 0, 1)

            projected_uv = pixel_coords[valid_mask].detach().cpu().numpy()
            # plt.figure(figsize=(5, 5))
            # plt.imshow(img_np)
            # plt.scatter(projected_uv[:, 0], projected_uv[:, 1], s=2, c='red')
            # plt.title(f"Batch {batch_idx}, Sample {i}")
            # plt.axis("off")
            # plt.show()


Processing batches: 100%|██████████| 1381/1381 [09:25<00:00,  2.44it/s]


##### Pose estimation trainer

In [ ]:
class PoseEstimationTrainer:
    """Trainer class for the Pose Estimation model.
    """

    def __init__(self, model, train_loader, val_loader, device='cuda', config=None, experiment=None):
        """summary

        Args:
            model (torch model): Model to be trained
            train_loader (dataloader): train dataloader
            val_loader (dataloader): validation dataloader
            device (str, optional): cuda or cpu. Defaults to 'cuda'.
            config (dict, optional): configuration file. Defaults to None.
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.config = config or {}

        # Loss function e optimizer
        self.criterion = PoseLoss(alpha=1.0, beta=1.0)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max= config['num_epochs'], eta_min=1e-6)
        # self.grad_accum_steps = self.config.get('grad_accum_steps', 8)


        # Tracking delle metriche
        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        self.step = 0

        # # Log model architecture to wandb
        # if wandb.run is not None:
        #     wandb.watch(self.model, log="all", log_freq=50)

        self.experiment = experiment
        self.num_batches = len(self.train_loader)


    def train_epoch(self):
        """Train the model for one epoch.

        Returns:
            avg_loss (float): average total loss of the epoch
            avg_trans_loss (float): average translation loss of the epoch
            avg_rot_loss (float): average rotation loss of the epoch
        """
        self.model.train()
        total_loss = 0
        total_trans_loss = 0
        total_rot_loss = 0

        # Numero totale di batch nel dataloader
        # total_batches = len(self.train_loader)

        pbar = tqdm(self.train_loader, desc='Training')
        self.optimizer.zero_grad()

        for batch_idx, batch in enumerate(pbar):
            gt_trans = batch['translation']
            gt_rot = batch['quaternion']
            obj_id = batch['obj_id']

            # Forward pass
            pred_rot, pred_trans, pixel_predictions = self.model(batch)

            # Compute loss (valori originali, non normalizzati)
            loss, _, _ = self.criterion(pred_trans, pred_rot, gt_trans, gt_rot, obj_id)

            # Normalizza SOLO per il backward (accumulo gradiente)
            # loss_normalized = loss / self.grad_accum_steps
            loss.backward()

            # Accumula le loss originali per le statistiche
            total_loss += loss.item()
            # total_trans_loss += trans_loss.item()
            # total_rot_loss += rot_loss.item()

            # Step ogni grad_accum_steps o all'ultimo batch
            # if (batch_idx + 1) % self.grad_accum_steps == 0 or (batch_idx + 1) == total_batches:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.optimizer.zero_grad()

            # Logging (usa i valori originali)
            if self.experiment is not None:
                if batch_idx % 100 == 0:
                    self.experiment.log_metrics({
                        "batch_loss": loss.item(),  # Valore originale
                        # "batch_trans_loss": trans_loss.item(),  # Valore originale
                        # "batch_rot_loss": rot_loss.item(),  # Valore originale
                        "learning_rate": self.optimizer.param_groups[0]['lr'],
                        "step": self.step
                    })

            self.step += 1

            # Update progress bar (valori originali)
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                # 'Trans': f'{trans_loss.item():.4f}',
                # 'Rot': f'{rot_loss.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })

        # Calcola le medie sui batch totali (non sui step di accumulo)
        avg_loss = total_loss / self.num_batches
        avg_trans_loss = total_trans_loss / self.num_batches
        avg_rot_loss = total_rot_loss / self.num_batches

        return avg_loss, avg_trans_loss, avg_rot_loss

    def validate(self):
        """Validate the model on the validation set after each epoch.

        Returns:
            avg_loss (float): average total loss of the epoch
            avg_trans_loss (float): average translation loss of the epoch
            avg_rot_loss (float): average rotation loss of the epoch
        """
        self.model.eval()
        total_loss = 0
        total_trans_loss = 0
        total_rot_loss = 0
        num_batches = len(self.val_loader)

        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc='Validation'):
                # images = batch['cropped_img']
                gt_trans = batch['translation']
                gt_rot = batch['quaternion'] # ['quaternion'] per quaternion ['rotation']
                obj_id = batch['obj_id']

                # Forward pass
                pred_rot, pred_trans, pixel_predictions = self.model(batch)

                # Calcola loss
                loss, trans_loss, rot_loss = self.criterion(pred_trans, pred_rot, gt_trans, gt_rot, obj_id)

                total_loss += loss.item()
                # total_trans_loss += trans_loss.item()
                # total_rot_loss += rot_loss.item()

        avg_loss = total_loss / num_batches
        # avg_trans_loss = total_trans_loss / num_batches
        # avg_rot_loss = total_rot_loss / num_batches

        if self.experiment is not None:
            self.experiment.log_metrics({
                "val_loss": avg_loss,
                # "val_trans_loss": avg_trans_loss,
                # "val_rot_loss": avg_rot_loss,
                "epoch": len(self.train_losses)
            })

        return avg_loss, _, _,
        # avg_trans_loss, avg_rot_loss

    def train(self, num_epochs):
        """Train the model for the specified number of epochs.

        Args:
            num_epochs (int): number of epochs to train the model for
        """
        print(f"Starting training for {num_epochs} epochs...")

        if self.experiment is not None:
            with self.experiment.train():
              watch(self.model)

        for epoch in range(num_epochs):
              print(f'\nEpoch {epoch+1}/{num_epochs}')

              # Training
              train_loss, train_trans_loss, train_rot_loss = self.train_epoch()

              # Validation
              val_loss, val_trans_loss, val_rot_loss = self.validate()

              # Scheduler step
              self.scheduler.step()

              # Save metrics
              self.train_losses.append(train_loss)
              self.val_losses.append(val_loss)

              # Log epoch metrics to wandb
              if self.experiment is not None:
                  self.experiment.log_metrics({
                      "epoch": epoch + 1,
                      "train_loss": train_loss,
                      # "train_trans_loss": train_trans_loss,
                      # "train_rot_loss": train_rot_loss,
                      "val_loss": val_loss,
                      # "val_trans_loss": val_trans_loss,
                      # "val_rot_loss": val_rot_loss,
                  })

              print(f'Train Loss: {train_loss:.4f}')
              print(f'Val Loss: {val_loss:.4f}')
              # print(f'Train Loss: {train_loss:.4f} (Trans: {train_trans_loss:.4f}, Rot: {train_rot_loss:.4f})')
              # print(f'Val Loss: {val_loss:.4f} (Trans: {val_trans_loss:.4f}, Rot: {val_rot_loss:.4f})')

              # Save best model
              if val_loss < self.best_val_loss:
                  self.best_val_loss = val_loss

                  os.makedirs(f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/extension/", exist_ok=True)

                  lr = self.optimizer.param_groups[0]['lr']
                  batch_size = self.config.get('batch_size', 32)
                  best_model_path = (
                      f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/extension/{config['name_saved_file']}_{config['backbone']}" # quaternion
                      f"_bs{batch_size}.pth"
                  )

                  # Save the model
                  torch.save({
                      'epoch': epoch,
                      'model_state_dict': self.model.state_dict(),
                      'optimizer_state_dict': self.optimizer.state_dict(),
                      'train_loss': train_loss,
                      'val_loss': val_loss,
                      'config': self.config
                  }, best_model_path)

                  # Log model to wandb
                  if self.experiment is not None:
                      self.experiment.log_metric("best_val_loss", val_loss)

                  print(f'Saved best model with val_loss: {val_loss:.4f}')

        print("Training completed!")

    def plot_losses(self):
        """Plot the training and validation losses.
        """
        plt.figure(figsize=(12, 8))

        plt.subplot(2, 2, 1)
        plt.plot(self.train_losses, label='Train Loss', color='blue')
        plt.plot(self.val_losses, label='Validation Loss', color='red')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.grid(True)

        # Log plot to wandb
        if self.experiment is not None:
            self.experiment.log_image(image_data= plt, name= "Plot losses")

        plt.tight_layout()
        plt.show()

In [ ]:
class PoseLoss(nn.Module):
    """Loss function combines translation and rotation
        returns a weighted sum of the translation and rotation losses.
    """

    def __init__(self, alpha=1.0, beta=1.0):
        """Initialize alpha and beta values.

        Args:
            alpha (float, optional): Wieght (importance) to give to translation. Defaults to 1.0.
            beta (float, optional): Wieght (importance) to give to rotation. Defaults to 1.0.
        """
        super(PoseLoss, self).__init__()
        self.alpha = alpha  # peso per translation loss
        self.beta = beta    # peso per rotation loss
        self.models_dir = "/content/datasets/linemod/DenseFusion/Linemod_preprocessed/models"
        self.models_dict = self.load_models_points(self.models_dir)


    def forward(self, pred_trans, pred_rot, gt_trans, gt_rot, obj_id=None):
        """Initialize alpha and beta values.

          Args:
              pred_trans (matrix , optional): Predicted translation.
              pred_rot (matrix, optional): Predicted rotation.
              gt_trans (matrix, optional): Ground truth translation.
              gt_rot (matrix, optional): Ground truth rotation.
        """
        # Translation loss (MSE)
        # trans_loss = F.mse_loss(pred_trans, gt_trans)

        # Frobenius norm of the differences between matrices
        # rot_loss = F.mse_loss(pred_rot, gt_rot.view(-1, 4)) # view(-1,4) per quaternion, view(-1, 3, 3) per rotation
        # rot_loss = self.quaternion_loss(pred_rot, gt_rot.view(-1, 4)) # view(-1,4) per quaternion, view(-1, 3, 3) per rotation
        total_loss = self.dense_loss(pred_trans, pred_rot, gt_trans, gt_rot, obj_id)


        # Total Loss
        # total_loss = self.alpha * trans_loss + self.beta * rot_loss

        # return total_loss, trans_loss, rot_loss
        return total_loss

    def quaternion_loss(self, pred_q, gt_q):
        pred_q = F.normalize(pred_q, dim=-1)
        gt_q = F.normalize(gt_q, dim=-1)
        dot = torch.sum(pred_q * gt_q, dim=-1).abs()
        return 1 - dot.mean()

    def dense_loss(self, pred_trans, pred_rot, gt_trans, gt_rot, obj_id=None):

        pred_rot = pred_rot.detach().cpu().numpy()
        gt_rot = gt_rot.detach().cpu().numpy() # Also detach gt_rot for consistency if quaternion library is used

        # Now convert the arrays of quaternion objects to rotation matrices
        pred_rot = R.from_quat(pred_rot).as_matrix()
        gt_rot = R.from_quat(gt_rot).as_matrix()


        # Convert back to torch tensors on the correct device
        pred_rot_matrix = torch.tensor(pred_rot, dtype=torch.float32, device=device)
        gt_rot_matrix = torch.tensor(gt_rot, dtype=torch.float32, device=device)

        # Ensure obj_id is used correctly to retrieve model points for each item in the batch
        batch_size = pred_trans.shape[0]
        total_loss = 0

        for i in range(batch_size):
            current_obj_id_int = int(obj_id[i].item()) # Get the integer object ID for this sample
            # Convert integer ID to string key format (e.g., 2 -> "02")
            current_obj_id_str = f"{current_obj_id_int:02d}"

            if current_obj_id_str not in self.models_dict:
                print(f"Warning: Model points not found for object ID {current_obj_id_str}. Skipping loss calculation for this sample.")
                # Add a small loss or skip this sample based on desired behavior
                total_loss += torch.tensor(0.0, device=pred_trans.device) # Add zero loss
                continue

            model_points = self.models_dict[current_obj_id_str].to(pred_trans.device) # Ensure points are on the same device
            num_points = model_points.shape[0]

            # Transform points with the predicted pose for the current sample
            # Ensure translation is broadcast correctly over the points
            pred_points = torch.matmul(model_points, pred_rot_matrix[i].T) + pred_trans[i].reshape(1, 3) # [N, 3]

            # Transform points with the gt pose for the current sample
            gt_points = torch.matmul(model_points, gt_rot_matrix[i].T) + gt_trans[i].reshape(1, 3) # [N, 3]

            # Calculate ADD-S/ADD loss for the current sample
            diff = pred_points - gt_points
            add_loss_per_point = torch.linalg.norm(diff, dim=1) # Norm along the feature dimension
            add_loss_sample = torch.mean(add_loss_per_point) # Average error over points

            total_loss += add_loss_sample

        # Average loss over the batch
        avg_batch_loss = total_loss / batch_size

        return avg_batch_loss, _, _

    def load_models_points(self, models_dir):
        """Load the 3D model points (vertices) for the LINEMOD dataset in a dictionary {class_name: points}.

        Args:
            models_dir (_type_): path of the .ply files

        Returns:
            model_points_dict: dictionary of model points for each object class
        """
        model_points_dict = {}
        class_names = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15"]

        for obj_id in class_names:
            model_path = os.path.join(models_dir, f'obj_{obj_id}.ply')
            if os.path.exists(model_path):
                try:
                    # Carica il modello 3D
                    mesh = trimesh.load(model_path)

                    # Estrai punti dalla superficie o usa vertices
                    if hasattr(mesh, 'vertices') and mesh.vertices is not None:
                        points = torch.tensor(mesh.vertices/1000.0, dtype=torch.float32).to(device)
                        sample_points = fps(points, None, ratio=301/points.size(0), random_start=False)[:300]
                        points = points[sample_points]
                    else:
                        continue

                    # Check for NaN or infinite values
                    if torch.any(torch.isnan(points)) or torch.any(torch.isinf(points)):
                        # Remove NaN/Inf points
                        valid_mask = ~(torch.any(torch.isnan(points), dim=1) | torch.any(torch.isinf(points), dim=1))
                        points = points[valid_mask].to(device)


                    if len(points) == 0:
                        print(f"No valid points found for object {obj_id}")
                        continue
                    model_points_dict[obj_id] = points

                except Exception as e:
                    print(f"Error loading model {model_path}: {e}")
                    continue

        return model_points_dict

In [ ]:
def plotPose(pathImage, translation_gt, rotation_gt, translation_pred, rotation_pred, experiment=None):

    image = cv2.imread(pathImage)
    transparent_image = image.copy()

    rotat_gt = rotation_gt.to(device).float()
    trans_gt = translation_gt.to(device).float()
    rotat_pred = rotation_pred.to(device).float()
    trans_pred = translation_pred.to(device).float()

    image_id = pathImage.split("/")[-1].split(".")[0]
    label = pathImage.split("/")[-3]
    camera_intrinsics = torch.tensor([572.4114, 0.0, 325.2611, 0.0, 573.57043, 242.04899, 0.0, 0.0, 1.0]).reshape(3, 3).to(device)

    meshModel = trimesh.load(f"/content/datasets/linemod/DenseFusion/Linemod_preprocessed/models/obj_{label}.ply")
    vertices = torch.tensor(meshModel.vertices / 1000, dtype=torch.float32).to(device)
    min_corner = vertices.min(dim=0).values
    max_corner = vertices.max(dim=0).values

    bounding_box_3d = torch.tensor([
        [min_corner[0], min_corner[1], min_corner[2]],
        [max_corner[0], min_corner[1], min_corner[2]],
        [max_corner[0], max_corner[1], min_corner[2]],
        [min_corner[0], max_corner[1], min_corner[2]],
        [min_corner[0], min_corner[1], max_corner[2]],
        [max_corner[0], min_corner[1], max_corner[2]],
        [max_corner[0], max_corner[1], max_corner[2]],
        [min_corner[0], max_corner[1], max_corner[2]],
    ], dtype=torch.float32).to(device)

    if rotat_gt.numel() == 4:
        rotat_gt = torch.tensor(
            quaternion.as_rotation_matrix(np.quaternion(*rotat_gt.cpu().numpy())),
            dtype=torch.float32
        ).to(device)
    else:
        rotat_gt = rotat_gt.reshape(3, 3)

    if rotat_pred.numel() == 4:
        rotat_pred = torch.tensor(
            quaternion.as_rotation_matrix(np.quaternion(*rotat_pred.cpu().numpy())),
            dtype=torch.float32
        ).to(device)
    else:
        rotat_pred = rotat_pred.reshape(3, 3)

    axes_3d = torch.tensor([
        [0, 0, 0],
        [0.15, 0, 0],
        [0, 0.15, 0],
        [0, 0, 0.15]
    ], dtype=torch.float32).to(device)

    axes_cam_gt = (rotat_gt @ axes_3d.T).T + trans_gt
    bounding_box_3d_cam_gt = (rotat_gt @ bounding_box_3d.T).T + trans_gt

    axes_2d_gt = (camera_intrinsics @ axes_cam_gt.T).T
    axes_2d_gt = axes_2d_gt[:, :2] / axes_2d_gt[:, 2:3]
    bounding_box_2d_gt = (camera_intrinsics @ bounding_box_3d_cam_gt.T).T
    bounding_box_2d_gt = (bounding_box_2d_gt[:, :2] / bounding_box_2d_gt[:, 2:3]).int()

    p_gt = [tuple(el.cpu().numpy()) for el in bounding_box_2d_gt]
    edges = [(0,1), (1,2), (2,3), (3,0), (0,4), (1,5), (2,6), (3,7), (4,5), (5,6), (6,7), (7,4)]
    for el in edges:
        cv2.line(image, p_gt[el[0]], p_gt[el[1]], (0,0,255), 5)

    p0_gt = tuple(axes_2d_gt[0].cpu().int().numpy())
    p1_gt = tuple(axes_2d_gt[1].cpu().int().numpy())
    p2_gt = tuple(axes_2d_gt[2].cpu().int().numpy())
    p3_gt = tuple(axes_2d_gt[3].cpu().int().numpy())

    cv2.arrowedLine(image, p0_gt, p1_gt, (0, 0, 255), 2)
    cv2.arrowedLine(image, p0_gt, p2_gt, (0, 255, 0), 2)
    cv2.arrowedLine(image, p0_gt, p3_gt, (255, 0, 0), 2)

    axes_cam_pred = (rotat_pred @ axes_3d.T).T + trans_pred
    bounding_box_3d_cam_pred = (rotat_pred @ bounding_box_3d.T).T + trans_pred

    axes_2d_pred = (camera_intrinsics @ axes_cam_pred.T).T
    axes_2d_pred = axes_2d_pred[:, :2] / axes_2d_pred[:, 2:3]
    bounding_box_2d_pred = (camera_intrinsics @ bounding_box_3d_cam_pred.T).T
    bounding_box_2d_pred = (bounding_box_2d_pred[:, :2] / bounding_box_2d_pred[:, 2:3]).int()

    p_pred = [tuple(el.cpu().numpy()) for el in bounding_box_2d_pred]
    for el in edges:
        cv2.line(image, p_pred[el[0]], p_pred[el[1]], (255, 0, 0), 5)

    p0_pred = tuple(axes_2d_pred[0].cpu().int().numpy())
    p1_pred = tuple(axes_2d_pred[1].cpu().int().numpy())
    p2_pred = tuple(axes_2d_pred[2].cpu().int().numpy())
    p3_pred = tuple(axes_2d_pred[3].cpu().int().numpy())

    cv2.arrowedLine(transparent_image, p0_pred, p1_pred, (0, 0, 255), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p2_pred, (0, 255, 0), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p3_pred, (255, 0, 0), 2)

    overlapImage = cv2.addWeighted(transparent_image, 0.5, image, 1, 0)
    img = cv2.cvtColor(overlapImage, cv2.COLOR_BGR2RGB)
    # plt.imshow(img)
    if experiment is not None:
      experiment.log_image(image_data=img, name= f"{label}_{image_id}")
    # plt.title("Object Pose Estimation (prediction is transparent)")
    # plt.show()

#### Training

In [ ]:
config = {
    "project_name": "pointnet",
    "experiment_name": "add_loss",
    "batch_size": 8,
    "num_epochs": 25,
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "backbone": "resnet18",
    "hidden_dim": 512,
    "img_size": 224,
    "alpha": 1.0,
    "beta": 1.0,
    "add_threshold": 0.1,
    "symmetric_objects": ["10"],
    "name_saved_file": "add_loss_",
    "geometric_dims" : [64,128,256],
    "fusion_dim" : 128
}

MODELS_DIR = "/content/datasets/linemod/DenseFusion/Linemod_preprocessed/models"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {DEVICE}")
print(f"Configuration: {config}")

# Modello
model = PoseEstimationPipeline(
         geometric_dims=config["geometric_dims"],
         fusion_dim=config['fusion_dim']
     ).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

experiment = comet_ml.start(
    api_key="qt32gGMiNUQ0MX8Kc1jhRdvlo",
    project_name=config['project_name'],
    experiment_config=comet_ml.ExperimentConfig(
        name=config["experiment_name"],
        parse_args=False)
)

experiment.log_parameters(config)

# # # --------------------------
# from torch.utils.data import DataLoader, Subset

# # Numero di campioni desiderati nel subset
# subset_size = 10
# subset_indices = list(range(subset_size))

# # Creiamo i subset dei dataset originali
# train_subset = Subset(train_loader.dataset, subset_indices)
# val_subset = Subset(val_loader.dataset, subset_indices)
# test_subset = Subset(test_loader.dataset, subset_indices)

# # Creiamo i nuovi DataLoader a partire dai subset
# train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True, collate_fn=pointcloud_collate_fn)
# val_loader = DataLoader(val_subset, batch_size=config["batch_size"], shuffle=False, collate_fn=pointcloud_collate_fn)
# test_loader = DataLoader(test_subset, batch_size=config["batch_size"], shuffle=False, collate_fn=pointcloud_collate_fn)
# # ----------------------

trainer = PoseEstimationTrainer(model, train_loader, val_loader, device=DEVICE, config=config, experiment= experiment)
trainer.train(num_epochs=config["num_epochs"])

!mkdir -p /content/checkpoints
!cp -r /content/drive/MyDrive/6D_pose_estimation/checkpoints /content


checkpoint = torch.load(f"/content/drive/MyDrive/6D_pose_estimation/checkpoints/extension/{config['name_saved_file']}_{config['backbone']}_bs{config['batch_size']}.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()


add_metric = ADDMetric(
    model=model,
    test_loader=test_loader,
    models_3D_dir=MODELS_DIR,
    symmetric_objects=config["symmetric_objects"],
    device=DEVICE,
    experiment= experiment,
    config=config
)


print("Evaluating with ADD metric...")
add_score, accuracy, detailed_results = add_metric.evaluate_model_with_add()


print(f"\nFinal Results:\nADD Score: {add_score:.4f}\nAccuracy: {accuracy:.4f}")

test_batch = next(iter(test_loader))

for idx, batch in enumerate(test_loader):
    gt_trans = batch['translation']
    gt_rot = batch['rotation']
    object_ids = batch['obj_id']
    sample_id = batch["sample_id"]

    with torch.no_grad():
      pred_rot, pred_trans, _ = model(batch)

      for i in range(len(object_ids)):
        if i == 0:
          img_path = f"/content/datasets/linemod/DenseFusion/Linemod_preprocessed/data/{sample_id[i][0]:02d}/rgb/{sample_id[i][1]:04d}.png"

          plotPose(img_path, gt_trans[i], gt_rot[i], pred_trans[i], pred_rot[i], experiment=experiment)
print(f"Plot salvati su comet_ml in project: {config['project_name']}, experiment: {config['experiment_name']}")

experiment.end()

Using device: cuda
Configuration: {'project_name': 'pointnet', 'experiment_name': 'add_loss', 'batch_size': 8, 'num_epochs': 25, 'learning_rate': 0.0001, 'weight_decay': 1e-05, 'backbone': 'resnet18', 'hidden_dim': 512, 'img_size': 224, 'alpha': 1.0, 'beta': 1.0, 'add_threshold': 0.1, 'symmetric_objects': ['10'], 'name_saved_file': 'add_loss_', 'geometric_dims': [64, 128, 256], 'fusion_dim': 128}
Total parameters: 12,620,720
Trainable parameters: 931,208
Starting training for 25 epochs...

Epoch 1/25


Validation: 100%|██████████| 296/296 [02:45<00:00,  1.78it/s]


Train Loss: 0.2800
Val Loss: 0.2064
Saved best model with val_loss: 0.2064

Epoch 2/25


Validation: 100%|██████████| 296/296 [02:46<00:00,  1.78it/s]


Train Loss: 0.1562
Val Loss: 0.2326

Epoch 3/25


...

Validation: 100%|██████████| 296/296 [02:46<00:00,  1.77it/s]

Train Loss: 0.1298
Val Loss: 0.1914
Training completed!


Evaluating with ADD metric...


Evaluating ADD: 100%|██████████| 296/296 [06:14<00:00,  1.26s/it]



Overall ADD: 0.1699
Overall Accuracy: 0.0017

Per-object results:
Object 06: ADD=0.1659, Acc=0.0000, Samples=177
Object 11: ADD=0.1667, Acc=0.0000, Samples=183
Object 15: ADD=0.1879, Acc=0.0000, Samples=184
Object 01: ADD=0.2007, Acc=0.0000, Samples=186
Object 04: ADD=0.1582, Acc=0.0000, Samples=180
Object 10: ADD=0.0844, Acc=0.0213, Samples=188
Object 09: ADD=0.1574, Acc=0.0000, Samples=188
Object 08: ADD=0.1858, Acc=0.0000, Samples=178
Object 05: ADD=0.1634, Acc=0.0000, Samples=179
Object 14: ADD=0.1945, Acc=0.0000, Samples=184
Object 02: ADD=0.1795, Acc=0.0000, Samples=182
Object 12: ADD=0.1784, Acc=0.0000, Samples=186
Object 13: ADD=0.1882, Acc=0.0000, Samples=173

Final Results:
ADD Score: 0.1699
Accuracy: 0.0017
Plot salvati su comet_ml in project: pointnet, experiment: add_loss


In [ ]:
import torch, gc

# Elimina variabili grandi
del model
del trainer
gc.collect()

# Libera memoria cache della GPU
torch.cuda.empty_cache()

# Forza la liberazione della memoria GPU (più aggressivo)
try:
    torch.cuda.ipc_collect()
except:
    pass
